http://www.infoq.com/cn/articles/deeplearning-tensorflow-actual-combat




In [ ]:
#创建模型的权重及偏置

weights = tf.Variable(tf.random_normal([784, 200], stddev=0.35), name="weights")

biases = tf.Variable(tf.zeros([200]), name="biases")



#指定变量所在设备为CPU:0

with tf.device("/cpu:0"):

  v = tf.Variable(...)



#初始化模型变量

init_op = tf.global_variables_initializer()

sess=tf.Session()

sess.run(init_op)



#保存模型变量，由三个文件组成model.data，model.index，model.meta

saver = tf.train.Saver()

saver.restore(sess, "/tmp/model")



#恢复模型变量

saver.restore(sess, "/tmp/model")





在复杂的深度学习模型中，存在大量的模型变量，并且期望能够一次性地初始化这些变量。TensorFlow提供了tf.variable_scope和tf.get_variable两个API，实现了共享模型变量。tf.get_variable(<name>, <shape>, <initializer>)：表示创建或返回指定名称的模型变量，其中name表示变量名称，shape表示变量的维度信息，initializer表示变量的初始化方法。tf.variable_scope(<scope_name>)：表示变量所在的命名空间，其中scope_name表示命名空间的名称。共享模型变量使用示例如下：

In [ ]:
#定义卷积神经网络运算规则，其中weights和biases为共享变量

def conv_relu(input, kernel_shape, bias_shape):

    # 创建变量"weights".

    weights = tf.get_variable("weights", kernel_shape, initializer=tf.random_normal_initializer())

    # 创建变量 "biases".

    biases = tf.get_variable("biases", bias_shape, initializer=tf.constant_initializer(0.0))

    conv = tf.nn.conv2d(input, weights, strides=[1, 1, 1, 1], padding='SAME')

    return tf.nn.relu(conv + biases)



#定义卷积层，conv1和conv2为变量命名空间

with tf.variable_scope("conv1"):

    # 创建变量 "conv1/weights", "conv1/biases".

    relu1 = conv_relu(input_images, [5, 5, 32, 32], [32])

with tf.variable_scope("conv2"):

    # 创建变量 "conv2/weights", "conv2/biases".

    relu1 = conv_relu(relu1, [5, 5, 32, 32], [32])

#两个隐藏层，一个logits输出层

hidden1 = tf.nn.relu(tf.matmul(images, weights) + biases)

hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)

logits = tf.matmul(hidden2, weights) + biases



#损失方程，采用softmax交叉熵算法

cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits( logits, labels, name='xentropy')

loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')



#选定优化算法及定义训练操作

optimizer = tf.train.GradientDescentOptimizer(learning_rate)

global_step = tf.Variable(0, name='global_step', trainable=False)

train_op = optimizer.minimize(loss, global_step=global_step)



模型训练及模型验证示例如下：

#加载训练数据，并执行网络训练

for step in xrange(FLAGS.max_steps):

    feed_dict = fill_feed_dict(data_sets.train, images_placeholder, labels_placeholder)

    _, loss_value = sess.run([train_op, loss], feed_dict=feed_dict)



#加载测试数据，计算模型精确度

for step in xrange(steps_per_epoch):

    feed_dict = fill_feed_dict(data_set, images_placeholder, labels_placeholder)

    true_count += sess.run(eval_correct, feed_dict=feed_dict)

TensorFlow可视化技术主要分为两部分：TensorFlow摘要模型及TensorBoard可视化组件。在摘要模型中，需要把模型变量或样本数据转换为TensorFlow summary操作，然后合并summary操作，最后通过Summary Writer操作写入TensorFlow的事件日志。TensorBoard通过读取事件日志，进行相关摘要信息的可视化展示，主要包括：Scalar图、图片数据可视化、声音数据展示、图模型可视化，以及变量数据的直方图和概率分布图。TensorFlow可视化技术的关键流程如下所示：

In [ ]:
#定义变量及训练数据的摘要操作

tf.summary.scalar('max', tf.reduce_max(var))

tf.summary.histogram('histogram', var)

tf.summary.image('input', image_shaped_input, 10)



#定义合并变量操作，一次性生成所有摘要数据

merged = tf.summary.merge_all()



#定义写入摘要数据到事件日志的操作

train_writer = tf.train.SummaryWriter(FLAGS.log_dir + '/train', sess.graph)

test_writer = tf.train.SummaryWriter(FLAGS.log_dir + '/test')



#执行训练操作，并把摘要信息写入到事件日志

summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))

train_writer.add_summary(summary, i)



  #下载示例code，并执行模型训练

  python mnist_with_summaries.py



#启动TensorBoard，TensorBoard的UI地址为http://ip_address:6006

tensorboard --logdir=/path/to/log-directory

https://github.com/MindorksOpenSource/AndroidTensorFlowMNISTExample

https://github.com/MindorksOpenSource

https://github.com/MindorksOpenSource/AndroidTensorFlowMachineLearningExample


https://github.com/locuslab/icnn

https://github.com/locuslab

http://python-for-android.readthedocs.io/en/latest/

https://github.com/nok/sklearn-porter

http://cs.smith.edu/dftwiki/index.php/Tutorial:_TensorFlow_References#Soon_Hin_Khor.27s_Gentlest_Introduction_to_TensorFlow

https://jalammar.github.io/Supercharging-android-apps-using-tensorflow/

App Structure Walkthrough

http://blog.csdn.net/freedom098/article/details/52117330

 tensorflow实现KNN识别MNIST KNN算法算是最简单的机器学习算法之一了，这个算法最大的特点是没有训练过程，是一种懒惰学习，这种结构也可以在tensorflow实现。

KNN的最核心就是距离度量方式，官方例程给出的是L1范数的例子，我这里改成了L2范数，也就是我们常说的欧几里得距离度量，另外，虽然是叫KNN，意思是选取k个最接近的元素来投票产生分类，但是这里只是用了最近的那个数据的标签作为预测值了。

距离的定义:      距离衡量包括欧式距离、夹角余弦等。对于文本分类来说，使用余弦(cosine)来计算相似度就比欧式(Euclidean)距离更合适


In [8]:
import tensorflow as tf  
import numpy as np  
  
def loadMNIST():  
    from tensorflow.examples.tutorials.mnist import input_data  
    mnist = input_data.read_data_sets('MNIST_data',one_hot=True)  
    return mnist  
def KNN(mnist):  
    train_x,train_y = mnist.train.next_batch(5000)  
    test_x,test_y = mnist.train.next_batch(200)  
  
    xtr = tf.placeholder(tf.float32,[None,784])  
    xte = tf.placeholder(tf.float32,[784])  
    distance = tf.sqrt(tf.reduce_sum(tf.pow(tf.add(xtr,tf.negative(xte)),2),reduction_indices=1))  
  
    pred = tf.argmin(distance,0)  
  
    init = tf.initialize_all_variables()  
  
    sess = tf.Session()  
    sess.run(init)  
  
    right = 0  
    for i in range(200):  
        ansIndex = sess.run(pred,{xtr:train_x,xte:test_x[i,:]})  
        print 'prediction is ',np.argmax(train_y[ansIndex])  
        print 'true value is ',np.argmax(test_y[i])  
        if np.argmax(test_y[i]) == np.argmax(train_y[ansIndex]):  
            right += 1.0  
    accracy = right/200.0  
    print accracy  
  
if __name__ == "__main__":  
    #mnist = loadMNIST()  
    KNN(mnist)  

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.
prediction is  3
true value is  3
prediction is  8
true value is  8
prediction is  7
true value is  7
prediction is  9
true value is  9
prediction is  9
true value is  9
prediction is  0
true value is  0
prediction is  1
true value is  1
prediction is  1
true value is  1
prediction is  5
true value is  5
prediction is  2
true value is  2
prediction is  0
true value is  0
prediction is  3
true value is  3
prediction is  3
true value is  8
prediction is  4
true value is  4
prediction is  7
true value is  7
prediction is  5
true value is  5
prediction is  2
true value is  2
prediction is  6
true value is  6
prediction is  4
true value is  4
prediction is  7
true value is  7
prediction is  1
true value is  1
prediction is  1
t

https://my.oschina.net/u/3281376/blog/847370

kNN分类算法的评价
优点

    1.简单，易于理解，易于实现，无需估计参数，无需训练；
        适合对稀有事件进行分类；
    3.特别适合于多分类问题(multi-modal,对象具有多个类别标签)， kNN比SVM的表现要好。

缺点

    1.样本不平衡时，会使结果出现偏差
    2.计算量庞大
    3.由于不需要训练，所以算法的可控性比较差

改进策略

分类效率：事先对样本属性进行约简，删除对分类结果影响较小的属性，快速的得出待分类样本的类别。该算法比较适用于样本容量比较大的类域的自动分类，而那些样本容量较小的类域采用这种算法比较容易产生误分。

分类效果：采用权值的方法（和该样本距离小的邻居权值大）来改进，Han等人于2002年尝试利用贪心法，针对文件分类实做可调整权重的k最近邻居法WAkNN (weighted adjusted k nearest neighbor)，以促进分类效果；而Li等人于2004年提出由于不同分类的文件本身有数量上有差异，因此也应该依照训练集合中各种分类的文件数量，选取不同数目的最近邻居，来参与分类

In [10]:
import tensorflow as tf 
import numpy as np

#import tensorflow.examples.tutorials.mnist.input_data as input_data
#mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)     #下载并加载mnist数据

train_X, train_Y = mnist.train.next_batch(5000) # 5000 for training (nn candidates)
test_X, test_Y = mnist.test.next_batch(100)   # 200 for testing


tra_X = tf.placeholder("float", [None, 784])
te_X = tf.placeholder("float", [784])

# Nearest Neighbor calculation using L1 Distance
# Calculate L1 Distance
distance = tf.reduce_sum(tf.abs(tf.add(tra_X,tf.negative(te_X))), reduction_indices=1)
# Prediction: Get min distance index (Nearest neighbor)
pred = tf.arg_min(distance, 0)

accuracy = 0.

# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # loop over test data
    for i in range(len(test_X)):
        # Get nearest neighbor
        nn_index = sess.run(pred, feed_dict={tra_X: train_X, te_X: test_X[i, :]})
        # Get nearest neighbor class label and compare it to its true label
        print("Test", i, "Prediction:", np.argmax(train_Y[nn_index]), \
            "True Class:", np.argmax(test_Y[i]))
        # Calculate accuracy
        if np.argmax(train_Y[nn_index]) == np.argmax(test_Y[i]):
            accuracy += 1./len(test_X)
    print("Done!")
    print("Accuracy:", accuracy)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
('Test', 0, 'Prediction:', 6, 'True Class:', 6)
('Test', 1, 'Prediction:', 0, 'True Class:', 0)
('Test', 2, 'Prediction:', 5, 'True Class:', 5)
('Test', 3, 'Prediction:', 4, 'True Class:', 4)
('Test', 4, 'Prediction:', 9, 'True Class:', 9)
('Test', 5, 'Prediction:', 9, 'True Class:', 9)
('Test', 6, 'Prediction:', 2, 'True Class:', 2)
('Test', 7, 'Prediction:', 1, 'True Class:', 1)
('Test', 8, 'Prediction:', 9, 'True Class:', 9)
('Test', 9, 'Prediction:', 4, 'True Class:', 4)
('Test', 10, 'Prediction:', 8, 'True Class:', 8)
('Test', 11, 'Prediction:', 1, 'True Class:', 7)
('Test', 12, 'Prediction:', 3, 'True Class:', 3)
('Test', 13, 'Prediction:', 9, 'True Class:', 9)
('Test', 14, 'Prediction:', 7, 'True Class:', 7)
('Test', 15, 'Prediction:', 9, 'True Class:', 4)
('Test', 16, 'Prediction:', 4, 'True Class:', 4)
('Test', 17, 'Prediction:', 4, 'True Class:', 4)
('Test', 18, 'Prediction:', 4, 'True Class:', 9)
('Tes

In [7]:
train_x,train_y = mnist.train.next_batch(1)
print train_x
print "===================="
print train_y


[[ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0

 pandas使用get_dummies进行one-hot编码    http://blog.csdn.net/lujiandong1/article/details/52836051
 
 离散特征的编码分为两种情况：

1、离散特征的取值之间没有大小的意义，比如color：[red,blue],那么就使用one-hot编码

2、离散特征的取值有大小的意义，比如size:[X,XL,XXL],那么就使用数值的映射{X:1,XL:2,XXL:3

Using the get_dummies will create a new column for every unique string in a certain column:使用get_dummies进行one-hot编码

In [84]:
import pandas as pd  
df = pd.DataFrame([  
            ['green', 'M', 10.1, 'class1'],   
            ['red', 'L', 13.5, 'class2'],   
            ['blue', 'XL', 15.3, 'class1']])  
  
df.columns = ['color', 'size', 'prize', 'class label']  
  
size_mapping = {  
           'XL': 3,  
           'L': 2,  
           'M': 1}  
df['size'] = df['size'].map(size_mapping)  
  
class_mapping = {label:idx for idx,label in enumerate(set(df['class label']))}  
df['class label'] = df['class label'].map(class_mapping)  

r=pd.get_dummies(df)  
print r

   size  prize  class label  color_blue  color_green  color_red
0     1   10.1            1           0            1          0
1     2   13.5            0           0            0          1
2     3   15.3            1           1            0          0



1. 可以利用numpy的array来创建tensor,需要使用的函数是tensorflow里的convert_to_tensor函数。也可将tensor转为numpy数据用eval函数, Session.run(). t.eval() is a shortcut for calling tf.get_default_session().run(t).

2. reshape() 所以-1，就是缺省值，就是先以你们合适，到时总数除以你们几个的乘积，我该是几就是几

3. http://www.tensorfly.cn/tfdoc/api_docs/python/array_ops.html

4. http://www.tensorfly.cn/tfdoc/api_docs/python/framework.html#Graph.as_default
Tensor Transformations

Casting
tf.string_to_number(string_tensor, out_type=None, name=None)
tf.to_double(x, name='ToDouble')
tf.to_float(x, name='ToFloat')
tf.to_bfloat16(x, name='ToBFloat16')
tf.to_int32(x, name='ToInt32')
tf.to_int64(x, name='ToInt64')
tf.cast(x, dtype, name=None)
Shapes and Shaping
tf.shape(input, name=None)
tf.size(input, name=None)
tf.rank(input, name=None)
tf.reshape(tensor, shape, name=None)
tf.squeeze(input, squeeze_dims=None, name=None)
tf.expand_dims(input, dim, name=None)
Slicing and Joining
tf.slice(input_, begin, size, name=None)
tf.split(split_dim, num_split, value, name='split')
tf.tile(input, multiples, name=None)
tf.pad(input, paddings, name=None)
tf.concat(concat_dim, values, name='concat')
tf.pack(values, name='pack')
tf.unpack(value, num=None, name='unpack')
tf.reverse_sequence(input, seq_lengths, seq_dim, name=None)
tf.reverse(tensor, dims, name=None)
tf.transpose(a, perm=None, name='transpose')
tf.gather(params, indices, name=None)
tf.dynamic_partition(data, partitions, num_partitions, name=None)
tf.dynamic_stitch(indices, data, name=None)


In [82]:
import tensorflow as tf  
import numpy as np

x=np.array([[1001,1002,1003],[3,4,5]])
print x
tx = tf.convert_to_tensor(x ,dtype=tf.float32)
print  tx
print "=======1======"


x=np.array([[1001,1002,1003],[3,4,5]])
tx =tf.reshape(x,[2,3]) 
print tx
print "=======2======"   
    
    
    
t1 = [[1,2,3], [4,5,6]]  
t2 = [[7,8,9], [10,11,12]]  
x =tf.reshape(t1,[2,3])
print x
print "=======3======"
    
    

  # Build a dataflow graph.
c = tf.constant([[1.0, 2.0], [3.0, 4.0]])
d = tf.constant([[1.0, 1.0], [0.0, 1.0]])
e = tf.matmul(c, d)
# Construct a `Session` to execut the graph.
sess = tf.Session()
# Execute the graph and store the value that `e` represents in `result`.
result = sess.run(e)
print result    
    


c = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
print c.get_shape()
d = tf.constant([[1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0]])
print d.get_shape()




t1 = [[1, 2, 3], [4, 5, 6]]
t2 = [[7, 8, 9], [10, 11, 12]]
t=tf.concat([t1, t2],0)
print t
with sess.as_default():
    print(t.eval())
#==> [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]
t=tf.concat([t1, t2],1)
print t
with sess.as_default():
    print(t.eval())


print "=======4======"




  
tensor = tf.constant([1, 2, 3, 4, 5, 6, 7,8])  
tensorReshape = tf.reshape(tensor,[2,4])  
print(tensorReshape)  
tensorReshape = tf.reshape(tensor,[-1,2,2])  
print(tensorReshape)  

print "=======5======"





[[1001 1002 1003]
 [   3    4    5]]
Tensor("Const_162:0", shape=(2, 3), dtype=float32)
=======1======
Tensor("Reshape_74:0", shape=(2, 3), dtype=int64)
=======2======
Tensor("Reshape_75:0", shape=(2, 3), dtype=int32)
=======3======
[[ 1.  3.]
 [ 3.  7.]]
(2, 3)
(4, 2)
Tensor("concat_37:0", shape=(4, 3), dtype=int32)
[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]]
Tensor("concat_38:0", shape=(2, 6), dtype=int32)
[[ 1  2  3  7  8  9]
 [ 4  5  6 10 11 12]]
=======4======
Tensor("Reshape_76:0", shape=(2, 4), dtype=int32)
Tensor("Reshape_77:0", shape=(2, 2, 2), dtype=int32)
=======5======


In [11]:
import tensorflow as tf
with tf.Graph().as_default() as net2_graph:
# Create some variables.
  v1 = tf.Variable(11, name="v1")
  v2 = tf.Variable(22, name="v2")

# Add an op to initialize the variables.
  init_op = tf.initialize_all_variables()

# Add ops to save and restore all the variables.
  saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, save the
# variables to disk.
  with tf.Session() as sess:
    sess.run(init_op)
    print "v1 = ", v1.eval()
    print "v2 = ", v2.eval()
  # Save the variables to disk.
    save_path = saver.save(sess, "./model/model3.ckpt")
    print "Model saved in file: ", save_path
    
    



Instructions for updating:
Use `tf.global_variables_initializer` instead.
v1 =  11
v2 =  22
Model saved in file:  ./model/model3.ckpt


In [ ]:

! rm ./model/*

!  tree 



In [17]:
import tensorflow as tf

#tf.reset_default_graph()

with tf.Graph().as_default() as net1_graph:
# Create some variables.
  v1 = tf.Variable(0, name="v1")
  v2 = tf.Variable(0, name="v2")

# Add ops to save and restore all the variables.
  saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
  with tf.Session() as sess:
  # Restore variables from disk.
    saver.restore(sess, "./model/model3.ckpt")
    print "Model restored."
    print "v1 = ", v1.eval()
    print "v2 = ", v2.eval()
    v_ = sess.run(v1)
    print(v_)


Model restored.
v1 =  11
v2 =  22
11


In [18]:
import tensorflow as tf

tf.reset_default_graph()
v1 = tf.Variable(1.1, name="v1")
v2 = tf.Variable(1.2, name="v2")

init = tf.initialize_all_variables()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    print v2.eval(sess)
    save_path="/tmp/zhjmodel-1.ckpt"
    saver.save(sess,save_path)
    print "Model stored...."

Instructions for updating:
Use `tf.global_variables_initializer` instead.
1.2
Model stored....


In [ ]:
ls -l


In [ ]:
with tf.Session() as sess:
    sess.run(init)
    print v1.eval(sess)

In [20]:
tf.reset_default_graph()
v1 = tf.Variable(1.0, name="v1")
init = tf.initialize_all_variables()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    print v1.eval(sess)
    save_path="/tmp/zhjmodel-1.ckpt"
    saver.restore(sess, save_path)
    print("Model restored.")
    print sess.run(v1)
    print sess.run(v1)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
1.0
Model restored.
1.1
1.1


In [85]:
import tensorflow as tf  
import numpy as np  
  
# save to file  
W = tf.Variable([[1,2,3],[4,5,6]],dtype = tf.float32,name='weight')  
b = tf.Variable([[1,2,3]],dtype = tf.float32,name='biases')  
  
init = tf.initialize_all_variables()  
saver = tf.train.Saver()  
with tf.Session() as sess:  
        sess.run(init)  
        save_path = saver.save(sess,"my_net/save_net.ckpt")  
        print ("save to path:",save_path)  

Instructions for updating:
Use `tf.global_variables_initializer` instead.
('save to path:', 'my_net/save_net.ckpt')


In [86]:
! tree

.
├── model
│   ├── checkpoint
│   ├── model3.ckpt.data-00000-of-00001
│   ├── model3.ckpt.index
│   └── model3.ckpt.meta
├── my_net
│   ├── checkpoint
│   ├── save_net.ckpt.data-00000-of-00001
│   ├── save_net.ckpt.index
│   └── save_net.ckpt.meta
└── TF_zhj.ipynb

2 directories, 9 files


In [89]:
import tensorflow as tf  
import numpy as np  
tf.reset_default_graph()
W = tf.Variable(np.arange(6).reshape((2,3)),dtype = tf.float32,name='weight')  
b = tf.Variable(np.arange(3).reshape((1,3)),dtype = tf.float32,name='biases')  
  
saver = tf.train.Saver()  
with tf.Session() as sess:  
        saver.restore(sess,"my_net/save_net.ckpt")  
        print ("weights:",sess.run(W))  
        print ("biases:",sess.run(b))

('weights:', array([[ 1.,  2.,  3.],
       [ 4.,  5.,  6.]], dtype=float32))
('biases:', array([[ 1.,  2.,  3.]], dtype=float32))


http://blog.csdn.net/lujiandong1/article/details/53301994

训练的过程：

1、先设置isTrain=True,然后会保存模型,设置isTrain=False会将训练好的模型加载进来进行测试

2、train_steps：表示训练的次数，例子中使用100
3、checkpoint_steps：表示训练多少次保存一下checkpoints，例子中使用50
4、checkpoint_dir：表示checkpoints文件的保存路径，例子中使用当前路径

In [98]:
import tensorflow as tf  
import numpy as np  
  
isTrain = True  
train_steps = 100  
checkpoint_steps = 50  
checkpoint_dir = 'continus/'  
  
x = tf.placeholder(tf.float32, shape=[None, 1])  
y = 4 * x + 4  
  
w = tf.Variable(tf.random_normal([1], -1, 1))  
b = tf.Variable(tf.zeros([1]))  
y_predict = w * x + b  
  
  
loss = tf.reduce_mean(tf.square(y - y_predict))  
optimizer = tf.train.GradientDescentOptimizer(0.5)  
train = optimizer.minimize(loss)  
  

  
saver = tf.train.Saver()  # defaults to saving all variables - in this case w and b  
x_data = np.reshape(np.random.rand(10).astype(np.float32), (10, 1))  
  
with tf.Session() as sess:  
    sess.run(tf.initialize_all_variables())  
    if isTrain:  
        for i in xrange(train_steps):  
            sess.run(train, feed_dict={x: x_data})  
            print ('+++++++')  
            if (i + 1) % checkpoint_steps == 0:  
                saver.save(sess, checkpoint_dir + 'model.ckpt', global_step=i+1)  
                
    else:  
        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)  
        print ('-------') 
        if ckpt and ckpt.model_checkpoint_path:  
            saver.restore(sess, ckpt.model_checkpoint_path)  
        else:  
            pass  
        print(sess.run(w))  
        print(sess.run(b))  
        

Instructions for updating:
Use `tf.global_variables_initializer` instead.
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++


In [99]:
! tree


.
├── continus
│   ├── checkpoint
│   ├── model.ckpt-100.data-00000-of-00001
│   ├── model.ckpt-100.index
│   ├── model.ckpt-100.meta
│   ├── model.ckpt-50.data-00000-of-00001
│   ├── model.ckpt-50.index
│   └── model.ckpt-50.meta
├── model
│   ├── checkpoint
│   ├── model3.ckpt.data-00000-of-00001
│   ├── model3.ckpt.index
│   └── model3.ckpt.meta
├── my_net
│   ├── checkpoint
│   ├── save_net.ckpt.data-00000-of-00001
│   ├── save_net.ckpt.index
│   └── save_net.ckpt.meta
└── TF_zhj.ipynb

3 directories, 16 files


http://blog.csdn.net/lujiandong1/article/details/53385092
 tensorflow将训练好的模型freeze,即将权重固化到图里面,并使用该模型进行预测
 
 说明：对于freeze操作,我们需要定义输出结点的名字.因为网络其实是比较复杂的,定义了输出结点的名字,那么freeze的时候就只把输出该结点所需要的子图都固化下来,其他无关的就舍弃掉.因为我们freeze模型的目的是接下来做预测.所以,一般情况下,output_node_names就是我们预测的目标.
 
 
 
 

In [1]:
#-*- coding:utf-8 -*-  
import tensorflow as tf  
import numpy as np  
  
  
with tf.variable_scope('Placeholder'):  
    inputs_placeholder = tf.placeholder(tf.float32, name='inputs_placeholder', shape=[None, 10])  
    labels_placeholder = tf.placeholder(tf.float32, name='labels_placeholder', shape=[None, 1])  
  
with tf.variable_scope('NN'):  
    W1 = tf.get_variable('W1', shape=[10, 1], initializer=tf.random_normal_initializer(stddev=1e-1))  
    b1 = tf.get_variable('b1', shape=[1], initializer=tf.constant_initializer(0.1))  
    W2 = tf.get_variable('W2', shape=[10, 1], initializer=tf.random_normal_initializer(stddev=1e-1))  
    b2 = tf.get_variable('b2', shape=[1], initializer=tf.constant_initializer(0.1))  
  
    a = tf.nn.relu(tf.matmul(inputs_placeholder, W1) + b1)  
    a2 = tf.nn.relu(tf.matmul(inputs_placeholder, W2) + b2)  
  
    y = tf.div(tf.add(a, a2), 2)  
  
with tf.variable_scope('Loss'):  
    loss = tf.reduce_sum(tf.square(y - labels_placeholder) / 2)  
  
with tf.variable_scope('Accuracy'):  
    predictions = tf.greater(y, 0.5, name="predictions")  
    correct_predictions = tf.equal(predictions, tf.cast(labels_placeholder, tf.bool), name="correct_predictions")  
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))  
  
  
adam = tf.train.AdamOptimizer(learning_rate=1e-3)  
train_op = adam.minimize(loss)  
  
# generate_data  
inputs = np.random.choice(10, size=[10000, 10])  
labels = (np.sum(inputs, axis=1) > 45).reshape(-1, 1).astype(np.float32)  
print('inputs.shape:', inputs.shape)  
print('labels.shape:', labels.shape)  
  
  
test_inputs = np.random.choice(10, size=[100, 10])  
test_labels = (np.sum(test_inputs, axis=1) > 45).reshape(-1, 1).astype(np.float32)  
print('test_inputs.shape:', test_inputs.shape)  
print('test_labels.shape:', test_labels.shape)  
  
batch_size = 32  
epochs = 10  
  
batches = []  
print("%d items in batch of %d gives us %d full batches and %d batches of %d items" % (  
    len(inputs),  
    batch_size,  
    len(inputs) // batch_size,  
    batch_size - len(inputs) // batch_size,  
    len(inputs) - (len(inputs) // batch_size) * 32)  
)  
for i in range(len(inputs) // batch_size):  
    batch = [ inputs[batch_size*i:batch_size*i+batch_size], labels[batch_size*i:batch_size*i+batch_size] ]  
    batches.append(list(batch))  
if (i + 1) * batch_size < len(inputs):  
    batch = [ inputs[batch_size*(i + 1):],labels[batch_size*(i + 1):] ]  
    batches.append(list(batch))  
print("Number of batches: %d" % len(batches))  
print("Size of full batch: %d" % len(batches[0]))  
print("Size if final batch: %d" % len(batches[-1]))  
  
global_count = 0  
  
with tf.Session() as sess:  
#sv = tf.train.Supervisor()  
#with sv.managed_session() as sess:  
    sess.run(tf.initialize_all_variables())  
    for i in range(epochs):  
        for batch in batches:  
            # print(batch[0].shape, batch[1].shape)  
            train_loss , _= sess.run([loss, train_op], feed_dict={  
                inputs_placeholder: batch[0],  
                labels_placeholder: batch[1]  
            })  
            # print('train_loss: %d' % train_loss)  
  
            if global_count % 100 == 0:  
                acc = sess.run(accuracy, feed_dict={  
                    inputs_placeholder: test_inputs,  
                    labels_placeholder: test_labels  
                })  
                print('accuracy: %f' % acc)  
            global_count += 1  
  
    acc = sess.run(accuracy, feed_dict={  
        inputs_placeholder: test_inputs,  
        labels_placeholder: test_labels  
    })  
    print("final accuracy: %f" % acc)  
    #在session当中就要将模型进行保存  
    saver = tf.train.Saver()  
    last_chkp = saver.save(sess, 'freeze/graph.chkp')  
    #sv.saver.save(sess, 'results/graph.chkp')  
  
for op in tf.get_default_graph().get_operations():  
    print(op.name)  

('inputs.shape:', (10000, 10))
('labels.shape:', (10000, 1))
('test_inputs.shape:', (100, 10))
('test_labels.shape:', (100, 1))
10000 items in batch of 32 gives us 312 full batches and -280 batches of 16 items
Number of batches: 313
Size of full batch: 2
Size if final batch: 2
Instructions for updating:
Use `tf.global_variables_initializer` instead.
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
accuracy: 0.530000
final accuracy: 0.530000
Placeholder/inp

In [2]:
! tree

.
├── continus
│   ├── checkpoint
│   ├── model.ckpt-100.data-00000-of-00001
│   ├── model.ckpt-100.index
│   ├── model.ckpt-100.meta
│   ├── model.ckpt-50.data-00000-of-00001
│   ├── model.ckpt-50.index
│   └── model.ckpt-50.meta
├── freeze
│   ├── checkpoint
│   ├── graph.chkp.data-00000-of-00001
│   ├── graph.chkp.index
│   └── graph.chkp.meta
├── model
│   ├── checkpoint
│   ├── model3.ckpt.data-00000-of-00001
│   ├── model3.ckpt.index
│   └── model3.ckpt.meta
├── my_net
│   ├── checkpoint
│   ├── save_net.ckpt.data-00000-of-00001
│   ├── save_net.ckpt.index
│   └── save_net.ckpt.meta
└── TF_zhj.ipynb

4 directories, 20 files


说明：
.data:存放的是权重参数
.meta:存放的是图和metadata,metadata是其他配置的数据
如果想将我们的模型固化，让别人能够使用，我们仅仅需要的是图和参数，metadata是不需要的
二、综合上述几个文件,生成可以使用的模型的步骤如下：

1、恢复我们保存的图
2、开启一个Session，然后载入该图要求的权重
3、删除对预测无关的metadata
4、将处理好的模型序列化之后保存




In [7]:

import os, argparse  
import tensorflow as tf  
from tensorflow.python.framework import graph_util  
  
#dir = os.path.dirname(os.path.realpath(__file__))  
  
def freeze_graph(model_folder):  
    # We retrieve our checkpoint fullpath  
    checkpoint = tf.train.get_checkpoint_state(model_folder)  
    input_checkpoint = checkpoint.model_checkpoint_path  
      
    # We precise the file fullname of our freezed graph  
    absolute_model_folder = "/".join(input_checkpoint.split('/')[:-1])  
    output_graph = absolute_model_folder + "/frozen_model.pb"  
  
    # Before exporting our graph, we need to precise what is our output node  
    # this variables is plural, because you can have multiple output nodes  
    #freeze之前必须明确哪个是输出结点,也就是我们要得到推论结果的结点  
    #输出结点可以看我们模型的定义  
    #只有定义了输出结点,freeze才会把得到输出结点所必要的结点都保存下来,或者哪些结点可以丢弃  
    #所以,output_node_names必须根据不同的网络进行修改  
    output_node_names = "Accuracy/predictions"  
  
    # We clear the devices, to allow TensorFlow to control on the loading where it wants operations to be calculated  
    clear_devices = True  
      
    # We import the meta graph and retrive a Saver  
    saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)  
  
    # We retrieve the protobuf graph definition  
    graph = tf.get_default_graph()  
    input_graph_def = graph.as_graph_def()  
  
    #We start a session and restore the graph weights  
    #这边已经将训练好的参数加载进来,也即最后保存的模型是有图,并且图里面已经有参数了,所以才叫做是frozen  
    #相当于将参数已经固化在了图当中   
    with tf.Session() as sess:  
        saver.restore(sess, input_checkpoint)  
  
        # We use a built-in TF helper to export variables to constant  
        output_graph_def = graph_util.convert_variables_to_constants(  
            sess,   
            input_graph_def,   
            output_node_names.split(",") # We split on comma for convenience  
        )   
  
        # Finally we serialize and dump the output graph to the filesystem  
        with tf.gfile.GFile(output_graph, "wb") as f:  
            f.write(output_graph_def.SerializeToString())  
        print("%d ops in the final graph." % len(output_graph_def.node))  
  
  
if __name__ == '__main__':  
    #parser = argparse.ArgumentParser()  
    #parser.add_argument("--model_folder", type=str, help="Model folder to export")  
    #args = parser.parse_args()  
    #freeze_graph(args.model_folder)
    freeze_graph( 'freeze/')
    

INFO:tensorflow:Froze 4 variables.
Converted 4 variables to const ops.
20 ops in the final graph.


In [8]:
! tree


.
├── continus
│   ├── checkpoint
│   ├── model.ckpt-100.data-00000-of-00001
│   ├── model.ckpt-100.index
│   ├── model.ckpt-100.meta
│   ├── model.ckpt-50.data-00000-of-00001
│   ├── model.ckpt-50.index
│   └── model.ckpt-50.meta
├── freeze
│   ├── checkpoint
│   ├── frozen_model.pb
│   ├── graph.chkp.data-00000-of-00001
│   ├── graph.chkp.index
│   └── graph.chkp.meta
├── model
│   ├── checkpoint
│   ├── model3.ckpt.data-00000-of-00001
│   ├── model3.ckpt.index
│   └── model3.ckpt.meta
├── my_net
│   ├── checkpoint
│   ├── save_net.ckpt.data-00000-of-00001
│   ├── save_net.ckpt.index
│   └── save_net.ckpt.meta
└── TF_zhj.ipynb

4 directories, 21 files


加载freeze后的模型,注意该模型已经是包含图和相应的参数了.所以,我们不需要再加载参数进来.也即该模型加载进来已经是可以使用了.
说明：
1、在预测的过程中,当把freeze后的模型加载进来后,我们只需要定义好输入的tensor和目标tensor即可

2、在这里要注意一下tensor_name和ops_name,

注意prefix/Placeholder/inputs_placeholder仅仅是操作的名字,prefix/Placeholder/inputs_placeholder:0才是tensor的名字

x = graph.get_tensor_by_name('prefix/Placeholder/inputs_placeholder:0')一定要使用tensor的名字

In [13]:
#-*- coding:utf-8 -*-  
import argparse   
import tensorflow as tf  
  
def load_graph(frozen_graph_filename):  
    # We parse the graph_def file  
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:  
        graph_def = tf.GraphDef()  
        graph_def.ParseFromString(f.read())  
  
    # We load the graph_def in the default graph  
    with tf.Graph().as_default() as graph:  
        tf.import_graph_def(  
            graph_def,   
            input_map=None,   
            return_elements=None,   
            name="prefix",   
            op_dict=None,   
            producer_op_list=None  
        )  
    return graph  
  
if __name__ == '__main__':  
    #parser = argparse.ArgumentParser()  
    #parser.add_argument("--frozen_model_filename", default="freeze/frozen_model.pb", type=str, help="Frozen model file to import")  
    #args = parser.parse_args()  
    #加载已经将参数固化后的图  
    #graph = load_graph(args.frozen_model_filename)  
     graph = load_graph("freeze/frozen_model.pb") 
    # We can list operations  
    #op.values() gives you a list of tensors it produces  
    #op.name gives you the name  
    #输入,输出结点也是operation,所以,我们可以得到operation的名字  
     for op in graph.get_operations():  
        print(op.name,op.values())  
        # prefix/Placeholder/inputs_placeholder  
        # ...  
        # prefix/Accuracy/predictions  
    #操作有:prefix/Placeholder/inputs_placeholder  
    #操作有:prefix/Accuracy/predictions  
    #为了预测,我们需要找到我们需要feed的tensor,那么就需要该tensor的名字  
    #注意prefix/Placeholder/inputs_placeholder仅仅是操作的名字,prefix/Placeholder/inputs_placeholder:0才是tensor的名字  
     x = graph.get_tensor_by_name('prefix/Placeholder/inputs_placeholder:0')  
     y = graph.get_tensor_by_name('prefix/Accuracy/predictions:0')  
          
     with tf.Session(graph=graph) as sess:  
        y_out = sess.run(y, feed_dict={  
            x: [[3, 5, 7, 4, 5, 1, 1, 1, 1, 1]] # < 45  
        })  
        print(y_out) # [[ 0.]] Yay!  
     print ("finish") 

(u'prefix/Placeholder/inputs_placeholder', (<tf.Tensor 'prefix/Placeholder/inputs_placeholder:0' shape=<unknown> dtype=float32>,))
(u'prefix/NN/W1', (<tf.Tensor 'prefix/NN/W1:0' shape=(10, 1) dtype=float32>,))
(u'prefix/NN/W1/read', (<tf.Tensor 'prefix/NN/W1/read:0' shape=(10, 1) dtype=float32>,))
(u'prefix/NN/b1', (<tf.Tensor 'prefix/NN/b1:0' shape=(1,) dtype=float32>,))
(u'prefix/NN/b1/read', (<tf.Tensor 'prefix/NN/b1/read:0' shape=(1,) dtype=float32>,))
(u'prefix/NN/W2', (<tf.Tensor 'prefix/NN/W2:0' shape=(10, 1) dtype=float32>,))
(u'prefix/NN/W2/read', (<tf.Tensor 'prefix/NN/W2/read:0' shape=(10, 1) dtype=float32>,))
(u'prefix/NN/b2', (<tf.Tensor 'prefix/NN/b2:0' shape=(1,) dtype=float32>,))
(u'prefix/NN/b2/read', (<tf.Tensor 'prefix/NN/b2/read:0' shape=(1,) dtype=float32>,))
(u'prefix/NN/MatMul', (<tf.Tensor 'prefix/NN/MatMul:0' shape=(?, 1) dtype=float32>,))
(u'prefix/NN/add', (<tf.Tensor 'prefix/NN/add:0' shape=(?, 1) dtype=float32>,))
(u'prefix/NN/Relu', (<tf.Tensor 'prefix/NN/

要获取图中ops的名字和对应的tensor的名字,可用如下的代码：

In [14]:
# We can list operations  
#op.values() gives you a list of tensors it produces  
#op.name gives you the name  
#输入,输出结点也是operation,所以,我们可以得到operation的名字  
for op in graph.get_operations():  
    print(op.name,op.values())  



(u'prefix/Placeholder/inputs_placeholder', (<tf.Tensor 'prefix/Placeholder/inputs_placeholder:0' shape=<unknown> dtype=float32>,))
(u'prefix/NN/W1', (<tf.Tensor 'prefix/NN/W1:0' shape=(10, 1) dtype=float32>,))
(u'prefix/NN/W1/read', (<tf.Tensor 'prefix/NN/W1/read:0' shape=(10, 1) dtype=float32>,))
(u'prefix/NN/b1', (<tf.Tensor 'prefix/NN/b1:0' shape=(1,) dtype=float32>,))
(u'prefix/NN/b1/read', (<tf.Tensor 'prefix/NN/b1/read:0' shape=(1,) dtype=float32>,))
(u'prefix/NN/W2', (<tf.Tensor 'prefix/NN/W2:0' shape=(10, 1) dtype=float32>,))
(u'prefix/NN/W2/read', (<tf.Tensor 'prefix/NN/W2/read:0' shape=(10, 1) dtype=float32>,))
(u'prefix/NN/b2', (<tf.Tensor 'prefix/NN/b2:0' shape=(1,) dtype=float32>,))
(u'prefix/NN/b2/read', (<tf.Tensor 'prefix/NN/b2/read:0' shape=(1,) dtype=float32>,))
(u'prefix/NN/MatMul', (<tf.Tensor 'prefix/NN/MatMul:0' shape=(?, 1) dtype=float32>,))
(u'prefix/NN/add', (<tf.Tensor 'prefix/NN/add:0' shape=(?, 1) dtype=float32>,))
(u'prefix/NN/Relu', (<tf.Tensor 'prefix/NN/

Training utilities
http://www.tensorfly.cn/tfdoc/api_docs/python/train.html#global_step

tf.train.write_graph(graph_def, logdir, name, as_text=True)

Writes a graph proto on disk.

The graph is written as a binary proto unless as_text is True.

v = tf.Variable(0, name='my_variable')
sess = tf.Session()
tf.train.write_graph(sess.graph_def, './write_graph', 'train.pbtxt')

Args:
graph_def: A GraphDef protocol buffer.
logdir: Directory where to write the graph.
name: Filename for the graph.
as_text: If True, writes the graph as an ASCII proto.


In [5]:
v = tf.Variable(0, name='my_variable')
sess = tf.Session()
tf.train.write_graph(sess.graph_def, './write_graph', 'train.pb',as_text=False)

'./write_graph/train.pb'

In [6]:
! tree


.
├── continus
│   ├── checkpoint
│   ├── model.ckpt-100.data-00000-of-00001
│   ├── model.ckpt-100.index
│   ├── model.ckpt-100.meta
│   ├── model.ckpt-50.data-00000-of-00001
│   ├── model.ckpt-50.index
│   └── model.ckpt-50.meta
├── freeze
│   ├── checkpoint
│   ├── frozen_model.pb
│   ├── graph.chkp.data-00000-of-00001
│   ├── graph.chkp.index
│   └── graph.chkp.meta
├── logs
│   └── events.out.tfevents.1491728464.haijunz-ThinkPad-T420
├── model
│   ├── checkpoint
│   ├── model3.ckpt.data-00000-of-00001
│   ├── model3.ckpt.index
│   └── model3.ckpt.meta
├── my_net
│   ├── checkpoint
│   ├── save_net.ckpt.data-00000-of-00001
│   ├── save_net.ckpt.index
│   └── save_net.ckpt.meta
├── TF_zhj.ipynb
└── write_graph
    ├── train.pb
    └── train.pbtxt

6 directories, 24 files


In [ ]:
v = tf.Variable(0, name='my_variable')
sess = tf.Session()
tf.train.write_graph(sess.graph_def, './write_graph', 'train.pbtxt')


meta_graph

https://github.com/tensorflow/tensorflow/blob/master/tensorflow/docs_src/programmers_guide/meta_graph.md

1.first build an inference graph, export it as a meta graph
2. Then later import it and extend it to a training graph.


In [6]:
import tensorflow as tf
import numpy as np  
import math
  
images = tf.constant(1.2, tf.float32, shape=[100, 28])
with tf.name_scope("hidden1"):
  weights = tf.Variable(
      tf.truncated_normal([28, 128],
                          stddev=1.0 / math.sqrt(float(28))),
      name="weights")
  biases = tf.Variable(tf.zeros([128]),
                       name="biases")
  hidden1 = tf.nn.relu(tf.matmul(images, weights) + biases)
# Hidden 2
with tf.name_scope("hidden2"):
  weights = tf.Variable(
      tf.truncated_normal([128, 32],
                          stddev=1.0 / math.sqrt(float(128))),
      name="weights")
  biases = tf.Variable(tf.zeros([32]),
                       name="biases")
  hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)
# Linear
with tf.name_scope("softmax_linear"):
  weights = tf.Variable(
      tf.truncated_normal([32, 10],
                          stddev=1.0 / math.sqrt(float(32))),
      name="weights")
  biases = tf.Variable(tf.zeros([10]),
                       name="biases")
  logits = tf.matmul(hidden2, weights) + biases
  tf.add_to_collection("logits", logits)

  init_all_op = tf.global_variables_initializer()
  #init_all_op =tf.initialize_all_variables()

with tf.Session() as sess:
  # Initializes all the variables.
  sess.run(init_all_op)
  # Runs to logit.
  sess.run(logits)
  # Creates a saver.
  saver0 = tf.train.Saver()
  saver0.save(sess, 'meta_graph/my-model-10000')
  # Generates MetaGraphDef.
  saver0.export_meta_graph('meta_graph/my-model-10000.meta')


In [7]:
with tf.Session() as sess:
  new_saver = tf.train.import_meta_graph('meta_graph/my-model-10000.meta')
  new_saver.restore(sess, 'meta_graph/my-model-10000')
  # Addes loss and train.
  labels = tf.constant(0, tf.int32, shape=[100], name="labels")
  batch_size = tf.size(labels)
  labels = tf.expand_dims(labels, 1)
  indices = tf.expand_dims(tf.range(0, batch_size), 1)
  concated = tf.concat([indices, labels], 1)
  onehot_labels = tf.sparse_to_dense(
      concated, tf.stack([batch_size, 10]), 1.0, 0.0)
  logits = tf.get_collection("logits")[0]
  cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
      labels=onehot_labels, logits=logits, name="xentropy")
  loss = tf.reduce_mean(cross_entropy, name="xentropy_mean")

  tf.summary.scalar('loss', loss)
  # Creates the gradient descent optimizer with the given learning rate.
  optimizer = tf.train.GradientDescentOptimizer(0.01)

  # Runs train_op.
  train_op = optimizer.minimize(loss)
  sess.run(train_op)

http://blog.csdn.net/qiqiaiairen/article/details/53184216




In [1]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.python.platform import gfile

data = np.arange(10,dtype=np.int32)

with tf.Session() as sess:
  print("# build graph and run")
  input1= tf.placeholder(tf.int32, [10], name="input")
  output1= tf.add(input1, tf.constant(100,dtype=tf.int32), name="output") #  data depends on the input data
  saved_result= tf.Variable(data, name="saved_result")
  do_save=tf.assign(saved_result,output1)
  
  os.system("rm -rf ./graph_data/")
  tf.train.write_graph(sess.graph_def, "./graph_data/", "test.pb", False) #proto
  #tf.initialize_all_variables()
  #r=tf.global_variables_initializer()
  # now set the data:
  result,_=sess.run([output1,do_save], {input1: data}) # calculate output1 and assign to 'saved_result'
  saver = tf.train.Saver(tf.all_variables())
  saver.save(sess,"./graph_data/checkpoint.data")

with tf.Session() as persisted_sess:
  print("load graph")
  with gfile.FastGFile("./graph_data/test.pb",'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    persisted_sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')
  print("map variables")
  persisted_result = persisted_sess.graph.get_tensor_by_name("saved_result:0")
  tf.add_to_collection(tf.GraphKeys.VARIABLES,persisted_result)
  try:
    saver = tf.train.Saver(tf.all_variables()) # 'Saver' misnomer! Better: Persister!
  except:pass
  print("load data")
  saver.restore(persisted_sess, "./graph_data/checkpoint.data")  # now OK
  print(persisted_result.eval())
  print("DONE")

# build graph and run
Instructions for updating:
Please use tf.global_variables instead.
load graph
map variables
Instructions for updating:
Please use tf.global_variables instead.
load data
[100 101 102 103 104 105 106 107 108 109]
DONE


http://stackoverflow.com/questions/34112202/tensorflow-checkpoint-save-and-read
https://sec.xiaomi.com/article/13

如何保存模型并在模型训练完后查看模型的训练参数？ 
TensorFlow的checkpoint机制使得其能够同时支持Online Learning和Continuous Learning，首先，通过tf.train.Saver()将训练好的或者训练过程中的模型保存成checkpoint:

_, loss_value, step = sess.run([train_op, loss, global_step])
saver.save(sess,"./checkpoint/checkpoint.ckpt", global_step=step)
然后通过restore()函数从本地的checkpoint文件中恢复模型，当然也可以从该点开始继续运行，也就是所谓的Continuous Learning：

ckpt = tf.train.get_checkpoint_state("./checkpoint/")
if ckpt and ckpt.model_checkpoint_path:
    print("Continue training from the model {}".format(ckpt.model_checkpoint_path))
    saver.restore(sess, ckpt.model_checkpoint_path)
    _, loss_value, step = sess.run([train_op, loss, global_step])

最后通过tf.trainable_variables()获取返回模型中所训练的参数：

for var in tf.trainable_varisbles():
    print var.name
    


In [ ]:
 

http://blog.csdn.net/lujiandong1/article/details/53222651
tensorflow中可视化loss,weight,bias,提供算法调试的信息

https://github.com/tensorflow/tensorflow/blob/master/tensorflow/tools/compatibility/tf_upgrade.py#L116
    
    
    

In [3]:
from __future__ import print_function  
import tensorflow as tf  
import numpy as np  
  
  
def add_layer(inputs, in_size, out_size, n_layer, activation_function=None):  
    # add one more layer and return the output of this layer  
    layer_name = 'layer%s' % n_layer  
    with tf.name_scope(layer_name):  
        with tf.name_scope('weights'):  
            Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W')  
            #tf.histogram_summary(layer_name + '/weights', Weights)  
            tf.summary.histogram(layer_name + '/weights', Weights) 
        with tf.name_scope('biases'):  
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')  
            #tf.histogram_summary(layer_name + '/biases', biases)  
            tf.summary.histogram(layer_name + '/biases', biases)
        with tf.name_scope('Wx_plus_b'):  
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)  
        if activation_function is None:  
            outputs = Wx_plus_b  
        else:  
            outputs = activation_function(Wx_plus_b, )  
        #tf.histogram_summary(layer_name + '/outputs', outputs)  
        tf.summary.histogram(layer_name + '/outputs', outputs)  
        return outputs  
  
  
# Make up some real data  
x_data = np.linspace(-1, 1, 300)[:, np.newaxis]  
noise = np.random.normal(0, 0.05, x_data.shape)  
y_data = np.square(x_data) - 0.5 + noise  
  
# define placeholder for inputs to network  
with tf.name_scope('inputs'):  
    xs = tf.placeholder(tf.float32, [None, 1], name='x_input')  
    ys = tf.placeholder(tf.float32, [None, 1], name='y_input')  
  
# add hidden layer  
l1 = add_layer(xs, 1, 10, n_layer=1, activation_function=tf.nn.relu)  
# add output layer  
prediction = add_layer(l1, 10, 1, n_layer=2, activation_function=None)  
  
# the error between prediciton and real data  
with tf.name_scope('loss'):  
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),  
                                        reduction_indices=[1]))  
    #tf.scalar_summary('loss', loss)  
    tf.summary.scalar('loss', loss) 
with tf.name_scope('train'):  
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)  
  
sess = tf.Session()  
#merged = tf.merge_all_summaries() 
merged=tf.summary.merge_all()
#writer = tf.train.SummaryWriter("logs/", sess.graph)  
writer =tf.summary.FileWriter("logs/", sess.graph) 
# important step  
sess.run(tf.initialize_all_variables())  
  
for i in range(1000):  
    sess.run(train_step, feed_dict={xs: x_data, ys: y_data})  
    if i % 50 == 0:  
        result = sess.run(merged,  
                          feed_dict={xs: x_data, ys: y_data})  
        writer.add_summary(result, i)  

Type is unsupported, or the types of the items don't match field type in CollectionDef.
unbound method to_proto() must be called with Variable instance as first argument (got Tensor instance instead)
Instructions for updating:
Use `tf.global_variables_initializer` instead.


AttributeError: 'Tensor' object has no attribute 'initializer'

 tensorflow构建网络模型
 http://blog.csdn.net/lujiandong1/article/details/53217248

In [22]:
from __future__ import print_function  
import tensorflow as tf  
import numpy as np  
import matplotlib.pyplot as plt  
  
def add_layer(inputs, in_size, out_size, activation_function=None):  
    Weights = tf.Variable(tf.random_normal([in_size, out_size]))  
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)  
    Wx_plus_b = tf.matmul(inputs, Weights) + biases  
    if activation_function is None:  
        outputs = Wx_plus_b  
    else:  
        outputs = activation_function(Wx_plus_b)  
    return outputs  
  
# Make up some real data  
x_data = np.linspace(-1, 1, 300)[:, np.newaxis]  
noise = np.random.normal(0, 0.05, x_data.shape)  
y_data = np.square(x_data) - 0.5 + noise  
  
##plt.scatter(x_data, y_data)  
##plt.show()  
  
# define placeholder for inputs to network  
xs = tf.placeholder(tf.float32, [None, 1])  
ys = tf.placeholder(tf.float32, [None, 1])  
# add hidden layer  
l1 = add_layer(xs, 1, 10, activation_function=tf.nn.relu)  
# add output layer  
prediction = add_layer(l1, 10, 1, activation_function=None)  
  
# the error between prediciton and real data  
loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys-prediction), reduction_indices=[1]))  
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)  
# important step  
init = tf.initialize_all_variables()  
sess= tf.Session()  
sess.run(init)  
  
for i in range(1000):  
    # training  
    sess.run(train_step, feed_dict={xs: x_data, ys: y_data})  
    if i % 50 == 0:  
        # to see the step improvement  
        print(sess.run(loss, feed_dict={xs: x_data, ys: y_data}))  

Instructions for updating:
Use `tf.global_variables_initializer` instead.
0.177221
0.00867744
0.00643269
0.00523516
0.00458893
0.00429269
0.00414355
0.00402482
0.00392423
0.00383714
0.00376122
0.00369587
0.00363715
0.00358822
0.00354582
0.0035086
0.0034749
0.0034466
0.0034179
0.00339362


https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/5-08-RNN2/
RNN LSTM 循环神经网络 (分类例子)

次我们会使用 RNN 来进行分类的训练 (Classification). 会继续使用到手写数字 MNIST 数据集. 让 RNN 从每张图片的第一行像素读到最后一行, 然后再进行分类判断. 接下来我们导入 MNIST 数据并确定 RNN 的各种参数(hyper-parameters):

接着是 cell 中的计算, 有两种途径:

使用 tf.nn.rnn(cell, inputs) (不推荐原因http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/). 但是如果使用这种方法, 可以参考这个代码(https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/recurrent_network.py);
使用 tf.nn.dynamic_rnn(cell, inputs) (推荐). 这次的练习将使用这种方式.
因 Tensorflow 版本升级原因, state_is_tuple=True 将在之后的版本中变为默认. 对于 lstm 来说, state可被分为(c_state, h_state).

如果使用tf.nn.dynamic_rnn(cell, inputs), 我们要确定 inputs 的格式. tf.nn.dynamic_rnn 中的 time_major 参数会针对不同 inputs 格式有不同的值.
1.如果 inputs 为 (batches, steps, inputs) ==> time_major=False;
2.如果 inputs 为 (steps, batches, inputs) ==> time_major=True;

最后是 output_layer 和 return 的值. 因为这个例子的特殊性, 有两种方法可以求得 results.

方式一: 直接调用final_state 中的 h_state (final_state[1]) 来进行运算

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.set_random_seed(1)   # set random seed

# 导入数据
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# hyperparameters
lr = 0.001                  # learning rate
training_iters = 100000     # train step 上限
batch_size = 128            
n_inputs = 28               # MNIST data input (img shape: 28*28)
n_steps = 28                # time steps
n_hidden_units = 128        # neurons in hidden layer
n_classes = 10              # MNIST classes (0-9 digits)
# x y placeholder
x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])

# 对 weights biases 初始值的定义
weights = {
    # shape (28, 128)
    'in': tf.Variable(tf.random_normal([n_inputs, n_hidden_units])),
    # shape (128, 10)
    'out': tf.Variable(tf.random_normal([n_hidden_units, n_classes]))
}
biases = {
    # shape (128, )
    'in': tf.Variable(tf.constant(0.1, shape=[n_hidden_units, ])),
    # shape (10, )
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes, ]))
}

def RNN(X, weights, biases):
    # 原始的 X 是 3 维数据, 我们需要把它变成 2 维数据才能使用 weights 的矩阵乘法
    # X ==> (128 batches * 28 steps, 28 inputs)
    X = tf.reshape(X, [-1, n_inputs])

    # X_in = W*X + b
    X_in = tf.matmul(X, weights['in']) + biases['in']
    # X_in ==> (128 batches, 28 steps, 128 hidden) 换回3维
    X_in = tf.reshape(X_in, [-1, n_steps, n_hidden_units])
    # 使用 basic LSTM Cell.
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units, forget_bias=1.0, state_is_tuple=True)
    init_state = lstm_cell.zero_state(batch_size, dtype=tf.float32) # 初始化全零 state
    outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, X_in, initial_state=init_state, time_major=False)
    results = tf.matmul(final_state[1], weights['out']) + biases['out']
    return results

pred = RNN(x, weights, biases)
#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
train_op = tf.train.AdamOptimizer(lr).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# init= tf.initialize_all_variables() # tf 马上就要废弃这种写法
# 替换成下面的写法:
init = tf.global_variables_initializer()



saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, save the
# variables to disk.
with tf.Session() as sess:
    sess.run(init)
    step = 0
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size, n_steps, n_inputs])
        sess.run([train_op], feed_dict={
            x: batch_xs,
            y: batch_ys,
        })
        if step % 20 == 0:
            print(sess.run(accuracy, feed_dict={
            x: batch_xs,
            y: batch_ys,
        }))
        step += 1
        save_path = saver.save(sess, "./model/model3.ckpt")
        print "Model saved in file: ", save_path

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
0.117188
Model saved in file:  ./model/model3.ckpt
Model saved in file:  ./model/model3.ckpt
Model saved in file:  ./model/model3.ckpt
Model saved in file:  ./model/model3.ckpt
Model saved in file:  ./model/model3.ckpt
Model saved in file:  ./model/model3.ckpt
Model saved in file:  ./model/model3.ckpt
Model saved in file:  ./model/model3.ckpt
Model saved in file:  ./model/model3.ckpt
Model saved in file:  ./model/model3.ckpt
Model saved in file:  ./model/model3.ckpt
Model saved in file:  ./model/model3.ckpt
Model saved in file:  ./model/model3.ckpt
Model saved in file:  ./model/model3.ckpt
Model saved in file:  ./model/model3.ckpt
Model saved in file:  ./model/model3.ckpt
Model saved in file:  ./model/model3.ckpt
Model saved in file:  ./model/model3.ckpt
Model saved in file:  ./model/model3.ckpt

https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/5-09-RNN3/
RNN LSTM (回归例子)这次我们会使用 RNN 来进行回归的训练 (Regression). 会继续使用到自己创建的 sin 曲线预测一条 cos 曲线. 接下来我们先确定 RNN 的各种参数(super-parameters):
tf.mul, tf.sub and tf.neg are deprecated in favor of tf.multiply, tf.subtract and tf.negative.

使用 Matplotlib 模块来进行可视化过程, 在建立好 model 以后, 设置 plt.ion() 使 plt.show()可以连续显示.



In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

BATCH_START = 0     # 建立 batch data 时候的 index
TIME_STEPS = 20     # backpropagation through time 的 time_steps
BATCH_SIZE = 50     
INPUT_SIZE = 1      # sin 数据输入 size
OUTPUT_SIZE = 1     # cos 数据输出 size
CELL_SIZE = 10      # RNN 的 hidden unit size 
LR = 0.006          # learning rate
def get_batch():
    global BATCH_START, TIME_STEPS
    # xs shape (50batch, 20steps)
    xs = np.arange(BATCH_START, BATCH_START+TIME_STEPS*BATCH_SIZE).reshape((BATCH_SIZE, TIME_STEPS)) / (10*np.pi)
    seq = np.sin(xs)
    res = np.cos(xs)
    BATCH_START += TIME_STEPS
    # returned seq, res and xs: shape (batch, step, input)
    return [seq[:, :, np.newaxis], res[:, :, np.newaxis], xs]
class LSTMRNN(object):
    def __init__(self, n_steps, input_size, output_size, cell_size, batch_size):
        self.n_steps = n_steps
        self.input_size = input_size
        self.output_size = output_size
        self.cell_size = cell_size
        self.batch_size = batch_size
        with tf.name_scope('inputs'):
            self.xs = tf.placeholder(tf.float32, [None, n_steps, input_size], name='xs')
            self.ys = tf.placeholder(tf.float32, [None, n_steps, output_size], name='ys')
        with tf.variable_scope('in_hidden'):
            self.add_input_layer()
        with tf.variable_scope('LSTM_cell'):
            self.add_cell()
        with tf.variable_scope('out_hidden'):
            self.add_output_layer()
        with tf.name_scope('cost'):
            self.compute_cost()
        with tf.name_scope('train'):
            self.train_op = tf.train.AdamOptimizer(LR).minimize(self.cost)
    def add_input_layer(self,):
        l_in_x = tf.reshape(self.xs, [-1, self.input_size], name='2_2D')  # (batch*n_step, in_size)
        # Ws (in_size, cell_size)
        Ws_in = self._weight_variable([self.input_size, self.cell_size])
        # bs (cell_size, )
        bs_in = self._bias_variable([self.cell_size,])
        # l_in_y = (batch * n_steps, cell_size)
        with tf.name_scope('Wx_plus_b'):
            l_in_y = tf.matmul(l_in_x, Ws_in) + bs_in
        # reshape l_in_y ==> (batch, n_steps, cell_size)
        self.l_in_y = tf.reshape(l_in_y, [-1, self.n_steps, self.cell_size], name='2_3D')
    def add_cell(self):
        lstm_cell = tf.contrib.rnn.BasicLSTMCell(self.cell_size, forget_bias=1.0, state_is_tuple=True)
        with tf.name_scope('initial_state'):
            self.cell_init_state = lstm_cell.zero_state(self.batch_size, dtype=tf.float32)
        self.cell_outputs, self.cell_final_state = tf.nn.dynamic_rnn(
            lstm_cell, self.l_in_y, initial_state=self.cell_init_state, time_major=False)
    def add_output_layer(self):
        # shape = (batch * steps, cell_size)
        l_out_x = tf.reshape(self.cell_outputs, [-1, self.cell_size], name='2_2D')
        Ws_out = self._weight_variable([self.cell_size, self.output_size])
        bs_out = self._bias_variable([self.output_size, ])
        # shape = (batch * steps, output_size)
        with tf.name_scope('Wx_plus_b'):
            self.pred = tf.matmul(l_out_x, Ws_out) + bs_out
    def compute_cost(self):
        losses = tf.contrib.legacy_seq2seq.sequence_loss_by_example(
            [tf.reshape(self.pred, [-1], name='reshape_pred')],
            [tf.reshape(self.ys, [-1], name='reshape_target')],
            [tf.ones([self.batch_size * self.n_steps], dtype=tf.float32)],
            average_across_timesteps=True,
            softmax_loss_function=self.ms_error,
            name='losses'
        )
        with tf.name_scope('average_cost'):
            self.cost = tf.div(
                tf.reduce_sum(losses, name='losses_sum'),
                self.batch_size,
                name='average_cost')
            tf.summary.scalar('cost', self.cost)

    def ms_error(self, y_pre, y_target):
        #return tf.square(tf.sub(y_pre, y_target))
        return tf.square(tf.subtract(y_pre, y_target))

    def _weight_variable(self, shape, name='weights'):
        initializer = tf.random_normal_initializer(mean=0., stddev=1.,)
        return tf.get_variable(shape=shape, initializer=initializer, name=name)

    def _bias_variable(self, shape, name='biases'):
        initializer = tf.constant_initializer(0.1)
        return tf.get_variable(name=name, shape=shape, initializer=initializer)
if __name__ == '__main__':
    # 搭建 LSTMRNN 模型
    model = LSTMRNN(TIME_STEPS, INPUT_SIZE, OUTPUT_SIZE, CELL_SIZE, BATCH_SIZE)
    sess = tf.Session()
    # 替换成下面的写法:
    #sess.run(tf.global_variables_initializer())
    # sess.run(tf.initialize_all_variables()) # tf 马上就要废弃这种写法
   # 替换成下面的写法:
    sess.run(tf.global_variables_initializer())
    plt.ion()   # 设置连续 plot
    plt.show()
    # 训练 200 次
    for i in range(200):
        seq, res, xs = get_batch()  # 提取 batch data
        if i == 0:
        # 初始化 data
            feed_dict = {
                    model.xs: seq,
                    model.ys: res,
            }
        else:
            feed_dict = {
                model.xs: seq,
                model.ys: res,
                model.cell_init_state: state    # 保持 state 的连续性
            }
        
        # 训练
        _, cost, state, pred = sess.run(
            [model.train_op, model.cost, model.cell_final_state, model.pred],
            feed_dict=feed_dict)
        
        # plotting
        plt.plot(xs[0, :], res[0].flatten(), 'r', xs[0, :], pred.flatten()[:TIME_STEPS], 'b--')
        plt.ylim((-1.2, 1.2))
        plt.draw()
        plt.pause(0.3)  # 每 0.3 s 刷新一次
        # 打印 cost 结果
        if i % 20 == 0:
            print('cost: ', round(cost, 4))

/usr/lib/pymodules/python2.7/matplotlib/backend_bases.py:2407: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


('cost: ', 18.2259)
('cost: ', 3.5294)
('cost: ', 1.4404)
('cost: ', 1.4068)
('cost: ', 0.1946)
('cost: ', 0.6116)


TclError: can't invoke "update" command: application has been destroyed

https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/5-12-scope/

在 Tensorflow 当中有两种途径生成变量 variable, 一种是 tf.get_variable(), 另一种是 tf.Variable(). 
如果在 tf.name_scope() 的框架下使用这两种方式, 结果会如下.
可以看出使用 tf.Variable() 定义的时候, 虽然 name 都一样, 但是为了不重复变量名, Tensorflow 输出的变量名并不是一样的. 所以, 本质上 var2, var21, var22 并不是一样的变量. 而另一方面, 使用tf.get_variable()定义的变量不会被tf.name_scope()当中的名字所影响.


In [2]:
import tensorflow as tf

with tf.name_scope("a_name_scope"):
    initializer = tf.constant_initializer(value=1)
    var1 = tf.get_variable(name='var1', shape=[1], dtype=tf.float32, initializer=initializer)
    var2 = tf.Variable(name='var2', initial_value=[2], dtype=tf.float32)
    var21 = tf.Variable(name='var2', initial_value=[2.1], dtype=tf.float32)
    var22 = tf.Variable(name='var2', initial_value=[2.2], dtype=tf.float32)


with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    print(var1.name)        # var1:0
    print(sess.run(var1))   # [ 1.]
    print(var2.name)        # a_name_scope/var2:0
    print(sess.run(var2))   # [ 2.]
    print(var21.name)       # a_name_scope/var2_1:0
    print(sess.run(var21))  # [ 2.0999999]
    print(var22.name)       # a_name_scope/var2_2:0
    print(sess.run(var22))  # [ 2.20000005]

Instructions for updating:
Use `tf.global_variables_initializer` instead.
var1:0
[ 1.]
a_name_scope/var2:0
[ 2.]
a_name_scope/var2_1:0
[ 2.0999999]
a_name_scope/var2_2:0
[ 2.20000005]


如果想要达到重复利用变量的效果, 我们就要使用 tf.variable_scope(), 并搭配 tf.get_variable() 这种方式产生和提取变量. 不像 tf.Variable() 每次都会产生新的变量, tf.get_variable() 如果遇到了同样名字的变量时, 它会单纯的提取这个同样名字的变量(避免产生新变量). 而在重复使用的时候, 一定要在代码中强调 scope.reuse_variables(), 否则系统将会报错, 以为你只是单纯的不小心重复使用到了一个变量.

In [3]:
with tf.variable_scope("a_variable_scope") as scope:
    initializer = tf.constant_initializer(value=3)
    var3 = tf.get_variable(name='var3', shape=[1], dtype=tf.float32, initializer=initializer)
    scope.reuse_variables()
    var3_reuse = tf.get_variable(name='var3',)
    var4 = tf.Variable(name='var4', initial_value=[4], dtype=tf.float32)
    var4_reuse = tf.Variable(name='var4', initial_value=[4], dtype=tf.float32)
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(var3.name)            # a_variable_scope/var3:0
    print(sess.run(var3))       # [ 3.]
    print(var3_reuse.name)      # a_variable_scope/var3:0
    print(sess.run(var3_reuse)) # [ 3.]
    print(var4.name)            # a_variable_scope/var4:0
    print(sess.run(var4))       # [ 4.]
    print(var4_reuse.name)      # a_variable_scope/var4_1:0
    print(sess.run(var4_reuse)) # [ 4.]

a_variable_scope/var3:0
[ 3.]
a_variable_scope/var3:0
[ 3.]
a_variable_scope/var4:0
[ 4.]
a_variable_scope/var4_1:0
[ 4.]


https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/5-12-scope/
RNN 例子的代码在这里, 整个 RNN 的结构已经在这里定义好了. 在 training RNN 和 test RNN 的时候, RNN 的 time_steps 会有不同的取值, 这将会影响到整个 RNN 的结构, 所以导致在 test 的时候, 不能单纯地使用 training 时建立的那个 RNN. 但是 training RNN 和 test RNN 又必须是有同样的 weights biases 的参数. 所以, 这时, 就是使用 reuse variable 的好时机.

首先定义training 和 test 的不同参数

In [ ]:
class TrainConfig:
    batch_size = 20
    time_steps = 20
    input_size = 10
    output_size = 2
    cell_size = 11
    learning_rate = 0.01


class TestConfig(TrainConfig):
    time_steps = 1
    
train_config = TrainConfig()
test_config = TestConfig()

然后让 train_rnn 和 test_rnn 在同一个 tf.variable_scope('rnn') 之下. 并且定义 scope.reuse_variables(), 使我们能把 train_rnn 的所有 weights, biases 参数全部绑定到 test_rnn 中. 这样, 不管两者的 time_steps 有多不同, 结构有多不同, train_rnn W, b 参数更新成什么样, test_rnn 的参数也更新成什么样.

In [ ]:
with tf.variable_scope('rnn') as scope:
    sess = tf.Session()
    train_rnn = RNN(train_config)
    scope.reuse_variables()
    test_rnn = RNN(test_config)
    sess.run(tf.global_variables_initializer())

Batch Normalization 批标准化
https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/5-13-BN/

atch normalization 是一种解决深度神经网络层数太多, 而没办法有效前向传递(forward propagate)的问题. 因为每一层的输出值都会有不同的 均值(mean) 和 方差(deviation), 所以输出数据的分布也不一样, 如下图, 从左到右是每一层的输入数据分布, 上排的没有 Batch normalization, 下排的有 Batch normalization.

我们以前说过, 为了更有效的学习数据, 我们会对数据预处理, 进行 normalization (请参考我制作的 为什么要特征标准化). 而现在请想象, 我们可以把 “每层输出的值” 都看成 “后面一层所接收的数据”. 对每层都进行一次 normalization 会不会更好呢? 这就是 Batch normalization 方法的由来.
https://morvanzhou.github.io/tutorials/machine-learning/ML-intro/3-08-batch-normalization/


In [4]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


ACTIVATION = tf.nn.relu # 每一层都使用 relu 
N_LAYERS = 7            # 一共7层隐藏层
N_HIDDEN_UNITS = 30     # 每个层隐藏层有 30 个神经元
def built_net(xs, ys, norm):
    def add_layer(inputs, in_size, out_size, activation_function=None):
        # 添加层功能
        Weights = tf.Variable(tf.random_normal([in_size, out_size], mean=0., stddev=1.))
        biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
        Wx_plus_b = tf.matmul(inputs, Weights) + biases
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
        return outputs

    fix_seed(1)

    layers_inputs = [xs]    # 记录每层的 input

    # loop 建立所有层
    for l_n in range(N_LAYERS):
        layer_input = layers_inputs[l_n]
        in_size = layers_inputs[l_n].get_shape()[1].value

        output = add_layer(
            layer_input,    # input
            in_size,        # input size
            N_HIDDEN_UNITS, # output size
            ACTIVATION,     # activation function
        )
        layers_inputs.append(output)    # 把 output 加入记录

    # 建立 output layer
    prediction = add_layer(layers_inputs[-1], 30, 1, activation_function=None)

    cost = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction), reduction_indices=[1]))
    train_op = tf.train.GradientDescentOptimizer(0.001).minimize(cost)
    return [train_op, cost, layers_inputs]


x_data = np.linspace(-7, 10, 500)[:, np.newaxis]
noise = np.random.normal(0, 8, x_data.shape)
y_data = np.square(x_data) - 5 + noise

# 可视化 input data
plt.scatter(x_data, y_data)
plt.show()



/usr/lib/pymodules/python2.7/matplotlib/collections.py:548: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


为了实现 Batch Normalization, 我们要对每一层的代码进行修改, 给 built_net 和 add_layer 都加上 norm 参数, 表示是否是 Batch Normalization 层:

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


ACTIVATION = tf.nn.relu # 每一层都使用 relu 
N_LAYERS = 7            # 一共7层隐藏层
N_HIDDEN_UNITS = 30     # 每个层隐藏层有 30 个神经元

def built_net(xs, ys, norm):
    def add_layer(inputs, in_size, out_size, activation_function=None, norm=False):


def built_net(xs, ys, norm):
    #def add_layer(inputs, in_size, out_size, activation_function=None):
    def add_layer(inputs, in_size, out_size, activation_function=None, norm=False):           
        # 添加层功能
        Weights = tf.Variable(tf.random_normal([in_size, out_size], mean=0., stddev=1.))
        biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
        Wx_plus_b = tf.matmul(inputs, Weights) + biases
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
        return outputs

    fix_seed(1)

    layers_inputs = [xs]    # 记录每层的 input

    # loop 建立所有层
    for l_n in range(N_LAYERS):
        layer_input = layers_inputs[l_n]
        in_size = layers_inputs[l_n].get_shape()[1].value

        output = add_layer(
            layer_input,    # input
            in_size,        # input size
            N_HIDDEN_UNITS, # output size
            ACTIVATION,     # activation function
        )
        layers_inputs.append(output)    # 把 output 加入记录

    # 建立 output layer
    prediction = add_layer(layers_inputs[-1], 30, 1, activation_function=None)

    cost = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction), reduction_indices=[1]))
    train_op = tf.train.GradientDescentOptimizer(0.001).minimize(cost)
    return [train_op, cost, layers_inputs]


x_data = np.linspace(-7, 10, 500)[:, np.newaxis]
noise = np.random.normal(0, 8, x_data.shape)
y_data = np.square(x_data) - 5 + noise

# 可视化 input data
plt.scatter(x_data, y_data)
plt.show()


In [ ]:
自编码 Autoencoder (非监督学习)


今天的代码，我们会运用两个类型：

第一，是通过Feature的压缩并解压，并将结果与原始数据进行对比，观察处理过后的数据是不是如预期跟原始数据很相像。（这里会用到MNIST数据）

在压缩环节：我们要把这个Features不断压缩，经过第一个隐藏层压缩至256个 Features，再经过第二个隐藏层压缩至128个。
在解压环节：我们将128个Features还原至256个，再经过一步还原至784个。
在对比环节：比较原始数据与还原后的拥有 784 Features 的数据进行 cost 的对比，根据 cost 来提升我的 Autoencoder 的准确率，下图是两个隐藏层的 weights 和 biases 的定义：


通过5个 Epoch 的训练，（通常情况下，想要得到好的的效果，我们应进行10 ~ 20个 Epoch 的训练）我们的结果如下：

上面一行是真实数据，下面一行是经过 encoder 和 decoder 之后的数据，如果继续进行训练，效果会更好。

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
tf.set_random_seed(1)   # set random seed
# 导入数据
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

X=mnist
# Parameter
learning_rate = 0.01
training_epochs = 5 # 五组训练
batch_size = 256
display_step = 1
examples_to_show = 10
# Network Parameters
n_input = 784  # MNIST data input (img shape: 28*28)


# hidden layer settings
n_hidden_1 = 256 # 1st layer num features
n_hidden_2 = 128 # 2nd layer num features


weights = {
	'encoder_h1':tf.Variable(tf.random_normal([n_input,n_hidden_1])),
	'encoder_h2': tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2])),
	'decoder_h1': tf.Variable(tf.random_normal([n_hidden_2,n_hidden_1])),
	'decoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
	}
biases = {
	'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
	'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
	'decoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
	'decoder_b2': tf.Variable(tf.random_normal([n_input])),
	}


#Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2
    
# Building the decoder
def decoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2


# Construct model
encoder_op = encoder(X) 			# 128 Features
decoder_op = decoder(encoder_op)	# 784 Features


# Prediction
y_pred = decoder_op	# After 
# Targets (Labels) are the input data.
y_true = X			# Before

# Define loss and optimizer, minimize the squared error
cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# Launch the graph
with tf.Session() as sess:
    # tf 马上就要废弃tf.initialize_all_variables()这种写法
    # 替换成下面:
    sess.run(tf.global_variables_initializer())
    total_batch = int(mnist.train.num_examples/batch_size)
    # Training cycle
    for epoch in range(training_epochs):
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)  # max(x) = 1, min(x) = 0
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={X: batch_xs})
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1),
                  "cost=", "{:.9f}".format(c))

    print("Optimization Finished!")

    # # Applying encode and decode over test set
    encode_decode = sess.run(
        y_pred, feed_dict={X: mnist.test.images[:examples_to_show]})
    # Compare original images with their reconstructions
    f, a = plt.subplots(2, 10, figsize=(10, 2))
    for i in range(examples_to_show):
        a[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
        a[1][i].imshow(np.reshape(encode_decode[i], (28, 28)))
    plt.show()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


TypeError: Expected binary or unicode string, got <tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7feea72556d0>

 tensorflow中使用LSTM去预测sinx函数
 
 http://blog.csdn.net/lujiandong1/article/details/53244699
 


第二，我们只看 encoder 压缩的过程，使用它将一个数据集压缩到只有两个Feature时，将数据放入一个二维坐标系内，特征压缩的效果如下

在类型二中，我们只显示 encoder 之后的数据， 并画在一个二维直角坐标系内。做法很简单，我们将原有 784 Features 的数据压缩成仅剩 2 Features 的数据

In [1]:


import tensorflow as tf  
import numpy as np  
import matplotlib.pyplot as plt  

  
BATCH_START = 0  
TIME_STEPS = 20  
BATCH_SIZE = 50  
INPUT_SIZE = 1  
OUTPUT_SIZE = 1  
CELL_SIZE = 10  
LR = 0.006  
  
  
def get_batch():  
    global BATCH_START, TIME_STEPS  
    # xs shape (50batch, 20steps)  
    xs = np.arange(BATCH_START, BATCH_START+TIME_STEPS*BATCH_SIZE).reshape((BATCH_SIZE, TIME_STEPS)) / (10*np.pi)  
    seq = np.sin(xs)  
    res = np.cos(xs)  
    BATCH_START += TIME_STEPS  
    # plt.plot(xs[0, :], res[0, :], 'r', xs[0, :], seq[0, :], 'b--')  
    # plt.show()  
    # returned seq, res and xs: shape (batch, step, input)  
    return [seq[:, :, np.newaxis], res[:, :, np.newaxis], xs]  
  
  
class LSTMRNN(object):  
    def __init__(self, n_steps, input_size, output_size, cell_size, batch_size):  
        self.n_steps = n_steps  
        self.input_size = input_size  
        self.output_size = output_size  
        self.cell_size = cell_size  
        self.batch_size = batch_size  
        with tf.name_scope('inputs'):  
            self.xs = tf.placeholder(tf.float32, [None, n_steps, input_size], name='xs')  
            self.ys = tf.placeholder(tf.float32, [None, n_steps, output_size], name='ys')  
        with tf.variable_scope('in_hidden'):  
            self.add_input_layer()  
        with tf.variable_scope('LSTM_cell'):  
            self.add_cell()  
        with tf.variable_scope('out_hidden'):  
            self.add_output_layer()  
        with tf.name_scope('cost'):  
            self.compute_cost()  
        with tf.name_scope('train'):  
            self.train_op = tf.train.AdamOptimizer(LR).minimize(self.cost)  
  
    def add_input_layer(self,):  
        l_in_x = tf.reshape(self.xs, [-1, self.input_size], name='2_2D')  # (batch*n_step, in_size)  
        # Ws (in_size, cell_size)  
        Ws_in = self._weight_variable([self.input_size, self.cell_size])  
        # bs (cell_size, )  
        bs_in = self._bias_variable([self.cell_size,])  
        # l_in_y = (batch * n_steps, cell_size)  
        with tf.name_scope('Wx_plus_b'):  
            l_in_y = tf.matmul(l_in_x, Ws_in) + bs_in  
        # reshape l_in_y ==> (batch, n_steps, cell_size)  
        self.l_in_y = tf.reshape(l_in_y, [-1, self.n_steps, self.cell_size], name='2_3D')  
  
    def add_cell(self):  
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(self.cell_size, forget_bias=1.0, state_is_tuple=True)
        #lstm_cell = tf.contrib.rnn.BasicLSTMCell(config.n_hidden, forget_bias=1.0)
        with tf.name_scope('initial_state'):  
            self.cell_init_state = lstm_cell.zero_state(self.batch_size, dtype=tf.float32)  
        self.cell_outputs, self.cell_final_state = tf.nn.dynamic_rnn(  
            lstm_cell, self.l_in_y, initial_state=self.cell_init_state, time_major=False)  
 







    def add_output_layer(self):  
        # shape = (batch * steps, cell_size)  
        l_out_x = tf.reshape(self.cell_outputs, [-1, self.cell_size], name='2_2D')  
        Ws_out = self._weight_variable([self.cell_size, self.output_size])  
        bs_out = self._bias_variable([self.output_size, ])  
        # shape = (batch * steps, output_size)  
        with tf.name_scope('Wx_plus_b'):  
            self.pred = tf.matmul(l_out_x, Ws_out) + bs_out  
  
    def compute_cost(self):  
        losses = tf.nn.seq2seq.sequence_loss_by_example(  
            [tf.reshape(self.pred, [-1], name='reshape_pred')],  
            [tf.reshape(self.ys, [-1], name='reshape_target')],  
            [tf.ones([self.batch_size * self.n_steps], dtype=tf.float32)],  
            average_across_timesteps=True,  
            softmax_loss_function=self.ms_error,  
            name='losses'  
        )  
        with tf.name_scope('average_cost'):  
            self.cost = tf.div(  
                tf.reduce_sum(losses, name='losses_sum'),  
                self.batch_size,  
                name='average_cost')  
            tf.scalar_summary('cost', self.cost)  
  
    def ms_error(self, y_pre, y_target):  
        return tf.square(tf.sub(y_pre, y_target))  
  
    def _weight_variable(self, shape, name='weights'):  
        initializer = tf.random_normal_initializer(mean=0., stddev=1.,)  
        return tf.get_variable(shape=shape, initializer=initializer, name=name)  
  
    def _bias_variable(self, shape, name='biases'):  
        initializer = tf.constant_initializer(0.1)  
        return tf.get_variable(name=name, shape=shape, initializer=initializer)  
  
  
if __name__ == '__main__':  
    model = LSTMRNN(TIME_STEPS, INPUT_SIZE, OUTPUT_SIZE, CELL_SIZE, BATCH_SIZE)  
    sess = tf.Session()  
    merged = tf.merge_all_summaries()  
    writer = tf.train.SummaryWriter("logs", sess.graph)  
    sess.run(tf.initialize_all_variables())  
    # relocate to the local dir and run this line to view it on Chrome (http://0.0.0.0:6006/):  
    # $ tensorboard --logdir='logs'  
  
    plt.ion()  
    plt.show()  
    for i in range(200):  
        seq, res, xs = get_batch()  
        if i == 0:  
            feed_dict = {  
                    model.xs: seq,  
                    model.ys: res,  
                    # create initial state  
            }  
        else:  
            feed_dict = {  
                model.xs: seq,  
                model.ys: res,  
                model.cell_init_state: state    # use last state as the initial state for this run  
            }  
  
        _, cost, state, pred = sess.run(  
            [model.train_op, model.cost, model.cell_final_state, model.pred],  
            feed_dict=feed_dict)  
  
        # plotting  
        # plt.plot(xs[0, :], res[0].flatten(), 'r', xs[0, :], pred.flatten()[:TIME_STEPS], 'b--')  
        # plt.ylim((-1.2, 1.2))  
        # plt.draw()  
        # plt.pause(0.3)  
  
        if i % 20 == 0:  
            print('cost: ', round(cost, 4))  
            result = sess.run(merged, feed_dict)  
            writer.add_summary(result, i)  

AttributeError: 'module' object has no attribute 'rnn_cell'

In [ ]:
import tensorflow as tf

# Create some variables.
v1 = tf.Variable(1, name="v1")
v2 = tf.Variable(2, name="v2")

# Add an op to initialize the variables.
init_op = tf.initialize_all_variables()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, save the
# variables to disk.
with tf.Session() as sess:
  sess.run(init_op)
  print "v1 = ", v1.eval()
  print "v2 = ", v2.eval()
  # Save the variables to disk.
  save_path = saver.save(sess, "/tmp/model-2.ckpt")
  print "Model saved in file: ", save_path

In [ ]:

tf.reset_default_graph()
# Create some variables.
v1 = tf.Variable(0, name="v1")
v2 = tf.Variable(0, name="v2")

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
  # Restore variables from disk.
  saver.restore(sess, "/tmp/model-2.ckpt")
  print "Model restored."
  print "v1 = ", v1.eval()
  print "v2 = ", v2.eval()

https://github.com/ZuzooVn/machine-learning-for-software-engineers/blob/master/README-zh-CN.md

自上而下的学习路线: 软件工程师的机器学习


# http://stackoverflow.com/questions/33759623/tensorflow-how-to-save-restore-a-model-python



In( and After) TensorFlow version 0.11.0RC1, you can save and restore your model directly by calling tf.train.export_meta_graph and tf.train.import_meta_graph according to https://www.tensorflow.org/programmers_guide/meta_graph

save model:

w1 = tf.Variable(tf.truncated_normal(shape=[10]), name='w1')
w2 = tf.Variable(tf.truncated_normal(shape=[20]), name='w2')
tf.add_to_collection('vars', w1)
tf.add_to_collection('vars', w2)
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver.save(sess, 'my-model')
# `save` method will call `export_meta_graph` implicitly.
# you will get saved graph files:my-model.meta
restore model:

sess = tf.Session()
new_saver = tf.train.import_meta_graph('my-model.meta')
new_saver.restore(sess, tf.train.latest_checkpoint('./'))
all_vars = tf.get_collection('vars')
for v in all_vars:
    v_ = sess.run(v)
    print(v_)

In [ ]:
w1 = tf.Variable(tf.truncated_normal(shape=[10]), name='w1')
w2 = tf.Variable(tf.truncated_normal(shape=[20]), name='w2')
tf.add_to_collection('vars', w1)
tf.add_to_collection('vars', w2)
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver.save(sess, 'my-model')

In [ ]:
sess = tf.Session()
new_saver = tf.train.import_meta_graph('my-model.meta')
new_saver.restore(sess, tf.train.latest_checkpoint('./'))
all_vars = tf.get_collection('vars')
for v in all_vars:
    v_ = sess.run(v)
    print(v_)

For TensorFlow version lower than 0.11.0RC1, the checkpoints that are saved contain values for the Variables in your model, not the model/graph itself, which means that the graph should be the same when you restore the checkpoint.

Here's an example for a linear regression where there's a training loop that saves variable checkpoints and an evaluation section that will restore variables saved in a prior run and compute predictions. Of course, you can also restore variables and continue training if you'd like.


https://www.tensorflow.org/api_docs/python/state_ops/variables#Variable




There are two parts to the model, the model definition, saved by Supervisor as graph.pbtxt in the model directory and the numerical values of tensors, saved into checkpoint files like model.ckpt-1003418.

The model definition can be restored using tf.import_graph_def, and the weights are restored using Saver.

However, Saver uses special collection holding list of variables that's attached to the model Graph, and this collection is not initialized using import_graph_def, so you can't use the two together at the moment (it's on our roadmap to fix). For now, you have to use approach of Ryan Sepassi -- manually construct a graph with identical node names, and use Saver to load the weights into it.

(Alternatively you could hack it by using by using import_graph_def, creating variables manually, and using tf.add_to_collection(tf.GraphKeys.VARIABLES, variable) for each variable, then using Saver)

As Yaroslav said, you can hack restoring from a graph_def and checkpoint by importing the graph, manually creating variables, and then using a Saver.

I implemented this for my personal use, so I though I'd share the code here.

Link: https://gist.github.com/nikitakit/6ef3b72be67b86cb7868

(This is, of course, a hack, and there is no guarantee that models saved this way will remain readable in future versions of TensorFlow.)



If it is an internally saved model, you just specify a restorer for all variables as

restorer = tf.train.Saver(tf.all_variables())
and use it to restore variables in a current session:

restorer.restore(self._sess, model_file)
For the external model you need to specify the mapping from the its variable names to your variable names. You can view the model variable names using the command

python /path/to/tensorflow/tensorflow/python/tools/inspect_checkpoint.py --file_name=/path/to/pretrained_model/model.ckpt
The inspect_checkpoint.py script can be found in './tensorflow/python/tools' folder of the Tensorflow source.

To specify the mapping, you can use my Tensorflow-Worklab, which contains a set of classes and scripts to train and retrain different models. It includes an example of retraining ResNet models, located here



## If it is an internally saved model, you just specify a restorer for all variables as

restorer = tf.train.Saver(tf.all_variables())
and use it to restore variables in a current session:

restorer.restore(self._sess, model_file)
For the external model you need to specify the mapping from the its variable names to your variable names. You can view the model variable names using the command

python /path/to/tensorflow/tensorflow/python/tools/inspect_checkpoint.py --file_name=/path/to/pretrained_model/model.ckpt

The inspect_checkpoint.py script can be found in './tensorflow/python/tools' folder of the Tensorflow source.

To specify the mapping, you can use my Tensorflow-Worklab, which contains a set of classes and scripts to train and retrain different models. It includes an example of retraining ResNet models, located here

You can also take this easier way.

Step.1 - Initialize all your variables

W1 = tf.Variable(tf.truncated_normal([6, 6, 1, K], stddev=0.1), name="W1")
B1 = tf.Variable(tf.constant(0.1, tf.float32, [K]), name="B1")

Similarly, W2, B2, W3, .....
Step.2 - Save the list inside Model Saver and Save it

model_saver = tf.train.Saver()

# Train the model and save it in the end
model_saver.save(session, "saved_models/CNN_New.ckpt")
Step. 3 - Restore the model

with tf.Session(graph=graph_cnn) as session:
    model_saver.restore(session, "saved_models/CNN_New.ckpt")
    print("Model restored.") 
    print('Initialized')
Step. 4 - Check Variable

W1 = session.run(W1)
print(W1)









# https://github.com/sdemyanov/tensorflow-worklab/

# http://stackoverflow.com/questions/37858866/how-to-restore-checkpoint-in-tensorflow-inside-ipython-or-anaconda


You need to reset the default graph at the beginning of your call when you run again the file.

If you don't reset the default graph, and run two times the line:

x = tf.Variable(1, name='x')
print x.name
You will see the first time that x has name "x:0" and the second time its name is "x_1:0". This is what confuses tf.train.Saver:

it first saves the value of x using name "x:0"
then in the next run you try to load the saved value of x, but now the name of the variable is "x_1:0", so the saver tries to load a saved value under the name "x_1:0" but cannot find it, and returns an error.
However, you can reset the default graph at the beginning using tf.reset_default_graph(). This will create an empty graph and use it as default graph.
Here the name of x can be the same in those two graphs:

# First run

tf.reset_default_graph()
x = tf.Variable(1, name='x')
print x.name  # prints 'x:0'

# Next run
tf.reset_default_graph()
x = tf.Variable(1, name='x')
print x.name  # prints 'x:0'
The two Variables can now have the same name because they are no longer in the same graph.

Another way of doing it is to create a graph at the beginning and use it as default graph:

graph = tf.Graph()
with graph.as_default():
    x = tf.Variable(1, name='x')

In [ ]:
# ############### tensor_save_named_vars.py#######################
tf.reset_default_graph()
v1 = tf.Variable(11, name="v1")
v2 = tf.Variable(2, name="v2")

# Add an op to initialize the variables.
init_op = tf.initialize_all_variables()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, save the
# variables to disk.
with tf.Session() as sess:
  sess.run(init_op)
  print "v1 = ", v1.eval()
  print "v2 = ", v2.eval()
  # Save the variables to disk.
  save_path = saver.save(sess, "/tmp/model3.ckpt")
  print "Model saved in file: ", save_path


In [ ]:

############################ tensor_restore.py######################

import tensorflow as tf
tf.reset_default_graph()
# Create some variables.
v1 = tf.Variable(0, name="v1")
v2 = tf.Variable(0, name="v2")

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
  # Restore variables from disk.
  saver.restore(sess, "/tmp/model3.ckpt")
  print "Model restored."
  print "v1 = ", v1.eval()
  print "v2 = ", v2.eval()


In [ ]:
ls -l /tmp/model3*

In [ ]:
sess = tf.Session()
new_saver = tf.train.import_meta_graph('/tmp/model3.ckpt.meta')
new_saver.restore(sess, tf.train.latest_checkpoint('/tmp/'))
all_vars = tf.get_collection('vars')
for v in all_vars:
    v_ = sess.run(v)
    print(v_)
    print v1.eval()
    


# Tensorflow Scikit Flow get GraphDef for Android (save *.pb file)

To save as pb file, you need to extract the graph_def from the constructed graph. You can do that as--

In [ ]:
from tensorflow.python.framework import tensor_shape, graph_util
from tensorflow.python.platform import gfile
sess = tf.Session()
final_tensor_name = 'results:0'     #Replace final_tensor_name with name of the final tensor in your graph
#########Build your graph and train########
## Your tensorflow code to build the graph
###########################################

outpt_filename = 'output_graph.pb'
output_graph_def = sess.graph.as_graph_def()
with gfile.FastGFile(outpt_filename, 'wb') as f:
  f.write(output_graph_def.SerializeToString())

In [ ]:
ls -l 

If you want to convert your trained variables to constants (to avoid using ckpt files to load the weights), you can use:

output_graph_def = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), [final_tensor_name])

In [ ]:
from tensorflow.python.framework import tensor_shape, graph_util
from tensorflow.python.platform import gfile
sess = tf.Session()
final_tensor_name = 'results:0'     #Replace final_tensor_name with name of the final tensor in your graph
#########Build your graph and train########
## Your tensorflow code to build the graph
###########################################

outpt_filename = 'output_graph.pb'
output_graph_def = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), [final_tensor_name])
with gfile.FastGFile(outpt_filename, 'wb') as f:
  f.write(output_graph_def.SerializeToString())

# Tensorflow GraphDef pb 文件读和写 
http://blog.csdn.net/eunicechen/article/details/51801351



In [ ]:
import tensorflow as tf
import os.path

from tensorflow.Python.platform import gfile
from google.protobuf import text_format

FLAGS = tf.app.flags.FLAGs

#Input Graph model file location
tf.app.flags.DEFINE_string('model_dir', '/home/eunice/data/test_code', """Paht to classify_image_graph_def.pb""")

#Output Graph model protobuf as text format & binary format
tf.app.flags.DEFINE_string('output_graph_txt', '/home/eunice/data/test_code/output_graph.pbtxt', """pbtxt""")
tf.app.flags.DEFINE_string('output_graph_pb', '/home/eunice/data/test_code/output_graph.pb', """pb""")

def convert_pb_to_pbtxt():
  model_filename = os.path.join(FLAGS.model_dir, 'classify_image_graph_def.pb')
  with gfile.FastGFile(model_filename, 'rb') as f:
   graph_def = tf.GraphDef()        #创建一个GraphDef
   graph_def.ParseFromString(f.read())   #ParseFromString(), reading message from protocol buffer binary fomat
  
  with gfile.FastGFile(FLAGS.output_graph_txt, 'wb') as f:
   f.write(text_format.MessageToString(graph_def))   # MessageToString(message, as_utf8=False, as_one_line=False)  Convert protobuf message to text format
  
#  with gfile.FastGFile(FLAGS.output_graph_pb, 'wb') as f:
#   f.write(graph_def.SerializeToString())  #serializes the message and returns it as a string. Note that the bytes 

#  tensorflow从0开始（6）——保存加载模型
http://blog.csdn.net/searobbers_duck/article/details/51721916


In [ ]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.python.platform import gfile


flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('summaries_dir', '/tmp/save_graph_logs', 'Summaries directory')

data = np.arange(10,dtype=np.int32)
with tf.Session() as sess:
  print("# build graph and run")
  input1= tf.placeholder(tf.int32, [10], name="input")
  output1= tf.add(input1, tf.constant(100,dtype=tf.int32), name="output") #  data depends on the input data
  saved_result= tf.Variable(data, name="saved_result")
  do_save=tf.assign(saved_result,output1)
  tf.initialize_all_variables()
  os.system("rm -rf /tmp/save_graph_logs")
  merged = tf.merge_all_summaries()
  train_writer = tf.train.SummaryWriter(FLAGS.summaries_dir,
                                        sess.graph)
  os.system("rm -rf /tmp/load")
  tf.train.write_graph(sess.graph_def, "/tmp/load", "test.pb", False) #proto
  # now set the data:
  result,_=sess.run([output1,do_save], {input1: data}) # calculate output1 and assign to 'saved_result'
  saver = tf.train.Saver(tf.all_variables())
  saver.save(sess,"checkpoint.data")

# 将TensorFlow的网络导出为单个文件发表于 2017-01-14 | 
http://encodets.me/2017/01/export-TensorFlow-network/

有时候，我们需要将TensorFlow的模型导出为单个文件（同时包含模型架构定义与权重），方便在其他地方使用（如在c++中部署网络）。利用tf.train.write_graph()默认情况下只导出了网络的定义（没有权重），而利用tf.train.Saver().save()导出的文件graph_def与权重是分离的，因此需要采用别的方法。

我们知道，graph_def文件中没有包含网络中的Variable值（通常情况存储了权重），但是却包含了constant值，所以如果我们能把Variable转换为constant，即可达到使用一个文件同时存储网络架构与权重的目标。

我们可以采用以下方式冻结权重并保存网络：



In [ ]:
import tensorflow as tf
from tensorflow.python.framework.graph_util import convert_variables_to_constants
# 构造网络
a = tf.Variable([[3],[4]], dtype=tf.float32, name='a')
b = tf.Variable(4, dtype=tf.float32, name='b')
# 一定要给输出tensor取一个名字！！
output = tf.add(a, b, name='out')
# 转换Variable为constant，并将网络写入到文件
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # 这里需要填入输出tensor的名字
    graph = convert_variables_to_constants(sess, sess.graph_def, ["out"])
    tf.train.write_graph(graph, '.', 'graph.pb', as_text=False)

In [ ]:
ls -l 

当恢复网络时，可以使用如下方式：

In [ ]:
import tensorflow as tf
with tf.Session() as sess:
    with open('./graph.pb', 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read()) 
        output = tf.import_graph_def(graph_def, return_elements=['out:0']) 
        print(sess.run(output))

可以看到之前的权重确实保存了下来!! ,  save as Txt



In [ ]:
import tensorflow as tf
from tensorflow.python.framework.graph_util import convert_variables_to_constants
a = tf.Variable([[3],[4]], dtype=tf.float32, name='a')
b = tf.Variable(4, dtype=tf.float32, name='b')
input_tensor = tf.placeholder(tf.float32, name='input')
output = tf.add((a+b), input_tensor, name='out')
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    graph = convert_variables_to_constants(sess, sess.graph_def, ["out"])
    tf.train.write_graph(graph, '.', 'graph.pb', as_text=True)

问题来了，我们的网络需要能有一个输入自定义数据的接口啊！不然这玩意有什么用。。别急，当然有办法

In [ ]:
import tensorflow as tf
from tensorflow.python.framework.graph_util import convert_variables_to_constants

tf.reset_default_graph()
a = tf.Variable([[3],[4]], dtype=tf.float32, name='a')
b = tf.Variable(4, dtype=tf.float32, name='b')
input_tensor = tf.placeholder(tf.float32, name='input')
output = tf.add((a+b), input_tensor, name='out')
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    graph = convert_variables_to_constants(sess, sess.graph_def, ["out"])
    tf.train.write_graph(graph, '.', 'graph.pb', as_text=False)

用上述代码重新保存网络至graph.pb，这次我们有了一个输入placeholder，下面来看看怎么恢复网络并输入自定义数据。

In [ ]:
import tensorflow as tf
tf.reset_default_graph()
with tf.Session() as sess:
    with open('./graph.pb', 'rb') as f: 
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read()) 
        output = tf.import_graph_def(graph_def, input_map={'input:0':4.}, return_elements=['out:0'], name='a') 
        print(sess.run(output))

可以看到结果没有问题，当然在input_map那里可以替换为新的自定义的placeholder，如下所示：看看输出，同样没有问题。

In [ ]:
import tensorflow as tf
new_input = tf.placeholder(tf.float32, shape=())
with tf.Session() as sess:
    with open('./graph.pb', 'rb') as f: 
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read()) 
        output = tf.import_graph_def(graph_def, input_map={'input:0':new_input}, return_elements=['out:0'], name='a') 
        print(sess.run(output, feed_dict={new_input:4}))

另外需要说明的一点是，在利用tf.train.write_graph写网络架构的时候，如果令as_text=True了，则在导入网络的时候，需要做一点小修改。



In [ ]:
import tensorflow as tf
from google.protobuf import text_format

with tf.Session() as sess:
	# 不使用'rb'模式
    with open('./graph.pb', 'r') as f:
        graph_def = tf.GraphDef()
        # 不使用graph_def.ParseFromString(f.read())
        text_format.Merge(f.read(), graph_def)
        output = tf.import_graph_def(graph_def, return_elements=['out:0']) 
        print(sess.run(output))

# ##参考资料http://stackoverflow.com/questions/34343259/is-there-an-example-on-how-to-generate-protobuf-files-holding-trained-tensorflow

# https://github.com/tensorflow/tensorflow/issues/616

# http://blog.csdn.net/searobbers_duck/article/details/51721916 
tensorflow从0开始（6）——保存加载模型

# 徐长卿学数据分析http://www.cnblogs.com/SSSR/p/5630534.html

# Is there an example on how to generate protobuf files holding trained Tensorflow graphs


http://stackoverflow.com/questions/34343259/is-there-an-example-on-how-to-generate-protobuf-files-holding-trained-tensorflow

I am looking at Google's example on how to deploy and use a pre-trained Tensorflow graph (model) on Android, at:

https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/android

This example uses a .pb file at: [this is a link to a file that downloads automatically] https://storage.googleapis.com/download.tensorflow.org/models/inception5h.zip

The example shows how to load the .pb file to a Tensorflow session and use it to perform classification, but it doesn't (?) mention how to generate such a .pb file after a graph is trained (e.g., in Python).

Are there any examples on how to do that?

